In [1]:
import json
import os
import re
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
from sklearn.cluster import KMeans 
import joblib
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import time

In [1]:
!pip install bayesian-optimization

You should consider upgrading via the 'c:\ana\python.exe -m pip install --upgrade pip' command.


In [3]:
df_list = []
with open("D:/predict_user_attribute20200911/raw_data/user_profile/position_data") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)

In [3]:
all_set = set([v.lower() for v in df[1]])

In [4]:
#企业创始人/CEO
ceo_set = set()
ceo_pattern = re.compile('|'.join(map(re.escape, ["董事长","总裁","ceo","董事","合伙人","总经理","副总经理","老板"])),flags=re.IGNORECASE)

In [5]:
#企业业务决策层
decision_level_set = set()
decision_level_pattern = re.compile('|'.join(map(re.escape, ['cio', 'cfo', 'coo', 'cto', 'cko', 'cpo', 'cgo', 'cmo', 'cso',"首席","行长","投资经理"])),flags=re.IGNORECASE)

In [6]:
#企业一般管理人员
manager_set = set()
manager_pattern = re.compile('|'.join(map(re.escape, ["总监","主管","总助","负责人","中层干部"])),flags=re.IGNORECASE)

In [7]:
# 事业单位员工
public_institution_set = set()
public_institution_pattern = re.compile('|'.join(map(re.escape, ["主任","教务","老师","教师","列车","政府","常委","教師","人民公仆","院长","医","驻华大使"])),flags=re.IGNORECASE)

In [8]:
# 高校学生
student_set = set()
student_pattern = re.compile('|'.join(map(re.escape, ["学生","研究生","博士","大学","应届生"])),flags=re.IGNORECASE)

In [9]:
# 其他（自由职业）
other_set = set()
other_pattern = re.compile('|'.join(map(re.escape, ["自由",])),flags=re.IGNORECASE) 

In [10]:
#企业普通员工
staff_set = set()
staff_pattern =  re.compile('|'.join(map(re.escape, ["工程师","民工","分析师","设计师","苦力","编程",
    "清洁","经理助理","客户经理","项目经理","从业者","规划师","部经理","公务员","销售经理"])),flags=re.IGNORECASE)

In [11]:
for v in all_set:
    if list(re.finditer(ceo_pattern,v)):
        ceo_set.add(v)
    elif list(re.finditer(decision_level_pattern,v)):
        decision_level_set.add(v)
    elif list(re.finditer(manager_pattern,v)):
        manager_set.add(v)
    elif list(re.finditer(public_institution_pattern,v)):
        public_institution_set.add(v)
    elif list(re.finditer(student_pattern,v)):
        student_set.add(v)
    else:
        if list(re.finditer(other_pattern,v)) or len(v) > 10:
            other_set.add(v)
        else:
            staff_set.add(v)

In [12]:
all_set = all_set - ceo_set - decision_level_set - manager_set - staff_set - public_institution_set - student_set - other_set
print(len(all_set))

0


In [13]:
print(len(ceo_set))
print(ceo_set)
print(len(decision_level_set))
print(decision_level_set)
print(len(manager_set))
print(manager_set)
print(len(public_institution_set))
print(public_institution_set)
print(len(student_set))
print(student_set)
print(len(other_set))
print(len(staff_set))

69
{'董事长兼司机', '董事总经理', '创业格子ceo', '合伙人', '品牌公关，营销传播，ceo-pr', '创新业务部总经理', '副总经理', '執行董事', '非总裁', '董事长/总裁/ceo/董事/合伙人', '市场部总经理', '高级副总裁', '总裁', '总经理', '华南副总经理', '总经理助理', '副总 & 合伙人', '战略投资管理中心副总经理', '高级合伙人', 'ceo与创始人', '产品经理，创业者，老板', '吉家集团全国发展副总经理', '老板', '小老板', '未来总经理', '干将、莫邪ceo，原双乾支付法人。', '创业者@合伙人', '执行总裁', '总经理，导演，视频设计师', 'ceo/董事/合伙人', '执行总经理', '梵溪花園品牌ceo', '高级副总裁 数字化转型顾问 区块链创新', '业务线总经理', '创始人/ceo', '城市总经理', '常务副总裁', '董事长兼ceo', '销售总经理', '创始ceo', '总经理/副总经理', '执行合伙人', '外贸公司总经理', '基金合伙人', '劲旅网副总裁', '创始人兼ceo', '不入流的总经理', 'ceo', '律所合伙人', '董事长秘书', '投资人 董事总经理', '首席执行官兼总经理', '董事长助理', '执行董事', '副总裁', '副总经理兼coo', '创始人&ceo', '创始合伙人', '创办人兼ceo', '创始人暨ceo', '北方区总经理', '总裁兼cgo', '渔你同行ceo', '董事长', '助理总裁', '投资银行部 助理董事', '老板娘', '公司创办人/ceo', '副ceo'}
31
{'联合创始人兼 cmo', '首席执行官', '首席策略官', 'coo', 'coo（首席运营官）', '帝国首席奇术师', '首席铲屎官', '首席搬砖官', '首席战略顾问', 'cto兼coo', 'cfo（首席财务官）', 'cpo', '投资经理', 'cto（首席技术官）', 'doctor', '高级投资经理', '首席顾问', '中国区首席执行官', 'cso', 'managing director', 'cio（首席信息官）', '首席律师', 'executive directo

In [14]:
def temp_function(x,ceo_set=ceo_set,decision_level_set=decision_level_set,manager_set=manager_set,
    public_institution_set=public_institution_set,staff_set=staff_set,student_set=student_set,other_set=other_set):
    #x = datetime.datetime.now().year - int(x.split("-")[0])
    y = 0
    if x in ceo_set:
        y = "企业创始人/CEO"
    elif x in decision_level_set:
        y = "企业业务决策层"
    elif x in manager_set:
        y = "企业一般管理人员"
    elif x in public_institution_set:
        y = "事业单位员工"
    elif x in staff_set:
        y = "企业普通员工"
    elif x in student_set:
        y = "高校学生"
    else:
        y = "其他（自由职业）"
    return y

In [15]:
df[1] = df[1].apply(lambda x : temp_function(x))

In [16]:
df[1].unique()

array(['其他（自由职业）', '企业普通员工', '企业一般管理人员', '企业创始人/CEO', '高校学生', '企业业务决策层',
       '事业单位员工'], dtype=object)

In [17]:
df.columns = ["aid","position",*["c{}".format(v) for v in range(10)]]
df

,aid,position,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,1,其他（自由职业）,380021,379528,380043,380126,379521,379529,379451,379526,378915,379489
1,10073,其他（自由职业）,349684,349389,349248,349729,347400,349431,349650,349540,349572,348987
2,10113,其他（自由职业）,378020,378036,377965,377980,378047,378021,377903,377873,378033,378025
3,10140,企业普通员工,381349,381108,381299,381252,380842,381326,379187,381311,381059,380919
4,101966,其他（自由职业）,381295,381108,381284,381313,381106,380842,381316,381279,381315,381310
...,...,...,...,...,...,...,...,...,...,...,...,...
13129,92106,企业普通员工,380768,380727,380481,375323,379814,380669,380484,379839,380482,381027
13130,93648,企业普通员工,381369,381411,381424,381338,381299,381425,381554,381393,381309,381280
13131,95954,企业普通员工,374274,373823,373982,373774,374270,374217,373884,374259,373955,374257
13132,97037,企业创始人/CEO,364501,373305,372948,373553,373574,373512,373551,373497,373480,373603


In [21]:
a_dict = {}
with open("../data/predict_user_attribute20200911/raw_data/user_profile/title_tokenize.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        a_dict[aid] = {}
        title_tokenize = temp_dict["title_tokenize"]
        a_dict[aid]["title_tokenize"] = title_tokenize

In [22]:
with open("../data/predict_user_attribute20200911/raw_data/user_profile/content_tokenize.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        if not aid in a_dict:
            print(aid)
            continue
        content_tokenize = temp_dict["content_tokenize"]
        a_dict[aid]["content_tokenize"] = content_tokenize

In [23]:
df

,aid,position,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,1,其他（自由职业）,380021,379528,380043,380126,379521,379529,379451,379526,378915,379489
1,10073,其他（自由职业）,349684,349389,349248,349729,347400,349431,349650,349540,349572,348987
2,10113,其他（自由职业）,378020,378036,377965,377980,378047,378021,377903,377873,378033,378025
3,10140,企业普通员工,381349,381108,381299,381252,380842,381326,379187,381311,381059,380919
4,101966,其他（自由职业）,381295,381108,381284,381313,381106,380842,381316,381279,381315,381310
...,...,...,...,...,...,...,...,...,...,...,...,...
13129,92106,企业普通员工,380768,380727,380481,375323,379814,380669,380484,379839,380482,381027
13130,93648,企业普通员工,381369,381411,381424,381338,381299,381425,381554,381393,381309,381280
13131,95954,企业普通员工,374274,373823,373982,373774,374270,374217,373884,374259,373955,374257
13132,97037,企业创始人/CEO,364501,373305,372948,373553,373574,373512,373551,373497,373480,373603


In [24]:
text_list = []
y_list = []
for index in df.index:
    y = df.loc[index,"position"]
    temp_text_list = []
    for i in range(10):
        c_text_list = []
        c = "c{}".format(i)
        aid = str(df.loc[index,c]).strip()
        if aid:
            if aid in a_dict:
                c_text_list = [*a_dict[aid]["title_tokenize"],"<PADDING>",*a_dict[aid]["content_tokenize"][:512]]
            else:
                print("aid不存在",index,c,aid)
        else:
            print(index)
        temp_text_list.append(c_text_list)     
    text_list.append(temp_text_list)
    y_list.append(y)

aid不存在 156 c3 130
aid不存在 386 c3 130
aid不存在 563 c8 161
aid不存在 662 c3 130
aid不存在 675 c3 130
aid不存在 720 c8 142
aid不存在 906 c2 161
aid不存在 1229 c0 169
aid不存在 1231 c6 154
aid不存在 1639 c4 130
aid不存在 1684 c6 134
aid不存在 1935 c1 108
aid不存在 3234 c2 130
aid不存在 3647 c8 96
aid不存在 3870 c5 130
aid不存在 4503 c6 130
aid不存在 5290 c8 126
aid不存在 5308 c3 130
aid不存在 5806 c5 130
aid不存在 5933 c2 213
aid不存在 5969 c5 130
aid不存在 6370 c1 130
aid不存在 6410 c7 134
aid不存在 8428 c9 96
aid不存在 8801 c2 161
aid不存在 9226 c1 130
aid不存在 11043 c5 154
aid不存在 11634 c2 184
aid不存在 11906 c9 142
aid不存在 12117 c3 130


In [26]:
with open("../data/predict_user_attribute20200911/raw_data/user_profile/position/原始最终数据集tokenize.jsonl","w") as f:
    for tag,content_tokenize in zip(y_list,text_list):
        f.write(json.dumps({"tag":tag,"all_content_tokenize":content_tokenize},ensure_ascii=False) + "\n")

In [3]:
import tensorflow as tf

def build_model(
    input_number:int,
    sentence_maxlen:int,
    vocab_size:int,
    tag_size:int,
    embedding_dim:int,
    embedding_matrix=None,
    is_embedding_training:bool=True,
    embedding_dropout_rate:float = 0.0,
    learning_rate = 1e-3,
    ):
    """建立模型
    input:
        sentence_maxlen : 句子的长度
        vocab_size : 词的个数
        tag_size : 分类的个数
        embedding_dim : word2vec训练时设置的向量长度
        embedding_matrix : word2vec词向量矩阵
        is_embedding_training : embedding层是否加入训练
        embedding_dropout_rate : embedding层dropout的比率
    """

    input_list =[ 
        tf.keras.layers.Input(shape=(sentence_maxlen,),name="input{}".format(i))
        for i in range(input_number)
    ]
    #embedding层
    if not (embedding_matrix is None):
        embedding_layer = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim = embedding_dim,
            embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
            trainable = is_embedding_training,
            input_length = sentence_maxlen,
        )
    else:
        embedding_layer = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim = embedding_dim,
            trainable = is_embedding_training,
            input_length = sentence_maxlen,
        )
    output_list = [
        embedding_layer(input_list[i]) for i in range(input_number)
    ]

    embedding_dropout_layer = tf.keras.layers.Dropout(embedding_dropout_rate)
    cnn_layer0 = tf.keras.layers.Conv1D(filters=128, kernel_size=10, strides=1, padding='valid',activation="tanh")
    cnn_layer1 = tf.keras.layers.MaxPool1D(2, padding='valid')
    cnn_layer2 = tf.keras.layers.Flatten()
    for layer in [embedding_dropout_layer,cnn_layer0,cnn_layer1,cnn_layer2]:
        output_list = [
            layer(output_list[i]) for i in range(input_number)
        ]
    output = tf.keras.layers.Concatenate()(output_list)
    
    # output = tf.keras.layers.Concatenate()(output_list)
    # output = tf.keras.layers.Dropout(embedding_dropout_rate)(output)
    # output = tf.keras.layers.Conv1D(filters=512, kernel_size=20, strides=1, padding='valid',activation="tanh")(output)
    # output = tf.keras.layers.MaxPool1D(2, padding='valid')(output)
    # output = tf.keras.layers.Flatten()(output)

    output = tf.keras.layers.Dropout(0.5)(output)
    #output = tf.keras.layers.Dense(16, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2())(output)

    if tag_size > 2:
        output = tf.keras.layers.Dense(tag_size, activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
        # output = tf.keras.layers.Dense(tag_size, activation='softmax',use_bias=True)(output)
        print("这是一个多分类模型")
    elif tag_size == 2:
        output = tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
        # output = tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True)(output)
        print("这是一个二分类模型")
    else:
        raise Exception("类别错误")

    model = tf.keras.Model(inputs=input_list, outputs=output, name='multi_textcnn')

    if tag_size > 2:
        metric_list = [
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
        model.compile(
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,
                clipnorm=1.0,
                clipvalue=0.5,
            ),
            metrics = metric_list,
        )
    else:
        metric_list = [
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.AUC(num_thresholds=10000)
        ]
        model.compile(
            loss = tf.keras.losses.BinaryCrossentropy(),
            optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3,
                clipnorm=1.0,
                clipvalue=0.5,
            ),
            metrics = metric_list,
        )
    return model


In [5]:
import os
import time
import collections
import sklearn.model_selection
import gensim
import numpy as np
import tensorflow as tf
#from utils import trans_gensim_word2vec2tf_embedding,trans_data2tf_data,trans_tokenize_data2tf_data,trans_multi_input_tokenize_data2npa
#from model_multi_textcnn import build_model

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
if not os.path.exists(os.path.join(APP_DIR,"data")):
    os.makedirs(os.path.join(APP_DIR,"data"))

def split_train_eval_test_dataset(dataset):
    """区分训练验证测试集
    """
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    print("总共有数据{}条".format(dataset_size))
    dataset = dataset.shuffle(dataset_size,seed=1)
    train_size = int(0.6 * dataset_size)
    eval_size = int(0.2 * dataset_size)
    test_size = int(0.2 * dataset_size)

    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)
    eval_dataset = test_dataset.skip(eval_size)
    test_dataset = test_dataset.take(test_size)
    return train_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
        eval_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
        test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

def split_train_eval_test_npa(x_npa,y_npa):
    x_train,x_test,y_train,y_test =  sklearn.model_selection.train_test_split(x_npa, y_npa, test_size=0.2, random_state=24)
    return x_train,y_train,x_test,y_test

def build_model_callback():
    callback_path = os.path.join(APP_DIR,"model_callback","weights.{epoch:02d}-{val_loss:.2f}.hdf5")
    if not os.path.exists(os.path.dirname(callback_path)):
        os.makedirs(os.path.dirname(callback_path))

    model_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=callback_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=False,
        save_weights_only=True,
        mode='auto',
        save_freq='epoch',
    )

    tf_board_dir = os.path.join(APP_DIR,"model_tensorboard")
    if not os.path.exists(os.path.dirname(tf_board_dir)):
         os.makedirs(os.path.dirname(tf_board_dir))

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tf_board_dir,histogram_freq=1,update_freq='batch')
    return model_callback,tensorboard_callback


if __name__ == "__main__":

    input_number = 10
    #句子的最大长度
    sentence_maxlen = 512
    #训练次数
    epochs = 7
    #批大小
    batch_size = 64
    #学习率
    learning_rate = 1e-3
    # learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    #     1e-3, decay_steps=100, decay_rate=0.1, staircase=False
    # )

    #embedding layer是否参加训练
    is_embedding_training = False
    # is_embedding_training = True
    #embedding的dropout比率
    #embedding_dropout_rate = 0.35
    embedding_dropout_rate = 0.2


    #数据文件
    #data_csv = os.path.join(APP_DIR,"data","最终数据集.csv")
    #data_jsonl = os.path.join(APP_DIR,"data","最终数据集tokenize.jsonl")
    #data_jsonl = "/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/age/最终数据集tokenize.jsonl"
    data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
    #word2vec路径
    #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
    word2vector_file_path = r"/home/zourui/data/dim256/word2vector.bin"
    #模型保存路径
    model_path = os.path.join(APP_DIR,"model_output1")
    ###以上是需要修改的部分

    if not os.path.exists(os.path.dirname(model_path)):
        os.makedirs(os.path.dirname(model_path))

    #导入gensim的word2vector
    embedding_matrix,word2vector_dict,word2index_dict = trans_gensim_word2vec2tf_embedding(word2vector_file_path)
    vocab_size,embedding_dim = embedding_matrix.shape

    #划分训练集+验证集，测试集
    #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
    x_npa,y_npa,tag2index_dict = trans_multi_input_tokenize_data2npa(data_jsonl,sentence_maxlen,word2index_dict)
    class_weight_dict = {tag:np.sqrt(len(y_npa)/number) for tag,number in enumerate(np.bincount(y_npa))}
    x_train_eval,y_train_eval,x_test,y_test = split_train_eval_test_npa(x_npa,y_npa)
    print(x_train_eval[:3])
    print(y_train_eval[:3])
    print(class_weight_dict)
    print(collections.Counter(y_train_eval))
    print(collections.Counter(y_test))
    tag_size = len(tag2index_dict)
    
     #建模模型
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
    model = build_model(
        input_number = input_number,
        sentence_maxlen = sentence_maxlen,
        vocab_size = vocab_size,
        tag_size = tag_size,
        embedding_dim = embedding_dim,
        embedding_matrix = embedding_matrix,
        is_embedding_training = is_embedding_training,
        embedding_dropout_rate = embedding_dropout_rate,
        learning_rate = learning_rate
    )

    #model_callback = build_model_callback()

    print(model.summary())

    history = model.fit(
        {"input{}".format(i):x_train_eval[:,i] for i in range(input_number)},
        y_train_eval,
        validation_split = 0.25,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        #callbacks=list(model_callback),
        #class_weight = class_weight_dict,
    )

    #测试集
    model.evaluate({"input{}".format(i):x_test[:,i] for i in range(input_number)},y_test)
    #保存模型
    print("即将保存模型：{}".format(model_path))
    model.save(model_path)

x_list[:1]:[[[64145, 60511, 11, 32049, 1674, 6, 129, 0, 31305, 9802, 2, 1, 11, 22136, 7621, 1518, 2, 28247, 341, 54, 12126, 11, 1006, 11786, 39276, 11, 1, 18, 1, 20828, 15290, 8973, 692, 11, 244, 341, 740, 8909, 112, 67716, 4256, 98, 11, 463, 2732, 129, 1583, 15550, 24570, 11, 21143, 19024, 389, 55, 14236, 57, 18, 11367, 54, 60511, 8, 14236, 6205, 11, 281, 1669, 19591, 245, 2897, 341, 740, 4256, 18, 40, 244, 20139, 11, 21560, 1036, 54, 5066, 274, 17043, 1380, 103, 8, 2361, 1, 18, 15357, 281, 128, 5649, 102, 20681, 8, 11, 54, 10361, 1333, 8479, 8, 33166, 18, 60511, 514, 71010, 76381, 8, 13568, 11, 32049, 4197, 6227, 12447, 3898, 6725, 103, 8, 291, 11, 991, 7631, 2585, 5203, 8, 2065, 18, 2789, 11367, 28422, 14236, 8, 15550, 24570, 2793, 11, 4155, 4450, 1507, 8, 6924, 1763, 11, 32049, 12552, 388, 3595, 1583, 2993, 2, 21421, 281, 883, 128, 6725, 103, 8, 1, 11, 991, 5203, 2502, 1, 60511, 11, 6, 281, 21560, 24034, 291, 1, 1, 18, 32049, 535, 12793, 3613, 50327, 11, 5649, 14521, 535, 5221, 151

KeyboardInterrupt: 

In [41]:
import os
import time
import collections
import sklearn.model_selection
import gensim
import numpy as np
import tensorflow as tf
import json
import gensim
import LAC
import pandas as pd
import sklearn.utils

class Position_model(object):
    
    def build_model(self,
        input_number:int,
        sentence_maxlen:int,
        vocab_size:int,
        tag_size:int,
        embedding_dim:int,
        embedding_matrix=None,
        is_embedding_training:bool=True,
        embedding_dropout_rate:float = 0.0,
        learning_rate = 1e-3,
        ):
        """建立模型
        input:
            sentence_maxlen : 句子的长度
            vocab_size : 词的个数
            tag_size : 分类的个数
            embedding_dim : word2vec训练时设置的向量长度
            embedding_matrix : word2vec词向量矩阵
            is_embedding_training : embedding层是否加入训练
            embedding_dropout_rate : embedding层dropout的比率
        """

        input_list =[ 
            tf.keras.layers.Input(shape=(sentence_maxlen,),name="input{}".format(i))
            for i in range(input_number)
        ]
        #embedding层
        if not (embedding_matrix is None):
            embedding_layer = tf.keras.layers.Embedding(
                input_dim = vocab_size,
                output_dim = embedding_dim,
                embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
                trainable = is_embedding_training,
                input_length = sentence_maxlen,
            )
        else:
            embedding_layer = tf.keras.layers.Embedding(
                input_dim = vocab_size,
                output_dim = embedding_dim,
                trainable = is_embedding_training,
                input_length = sentence_maxlen,
            )
        output_list = [
            embedding_layer(input_list[i]) for i in range(input_number)
        ]

        embedding_dropout_layer = tf.keras.layers.Dropout(embedding_dropout_rate)
        cnn_layer0 = tf.keras.layers.Conv1D(filters=128, kernel_size=10, strides=1, padding='valid',activation="tanh")
        cnn_layer1 = tf.keras.layers.MaxPool1D(2, padding='valid')
        cnn_layer2 = tf.keras.layers.Flatten()
        for layer in [embedding_dropout_layer,cnn_layer0,cnn_layer1,cnn_layer2]:
            output_list = [
                layer(output_list[i]) for i in range(input_number)
            ]
        output = tf.keras.layers.Concatenate()(output_list)

        # output = tf.keras.layers.Concatenate()(output_list)
        # output = tf.keras.layers.Dropout(embedding_dropout_rate)(output)
        # output = tf.keras.layers.Conv1D(filters=512, kernel_size=20, strides=1, padding='valid',activation="tanh")(output)
        # output = tf.keras.layers.MaxPool1D(2, padding='valid')(output)
        # output = tf.keras.layers.Flatten()(output)

        output = tf.keras.layers.Dropout(0.5)(output)
        #output = tf.keras.layers.Dense(16, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2())(output)

        if tag_size > 2:
            output = tf.keras.layers.Dense(tag_size, activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
            # output = tf.keras.layers.Dense(tag_size, activation='softmax',use_bias=True)(output)
            print("这是一个多分类模型")
        elif tag_size == 2:
            output = tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2())(output)
            # output = tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True)(output)
            print("这是一个二分类模型")
        else:
            raise Exception("类别错误")

        model = tf.keras.Model(inputs=input_list, outputs=output, name='multi_textcnn')

        if tag_size > 2:
            metric_list = [
                tf.keras.metrics.SparseCategoricalAccuracy(),
            ]
            model.compile(
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,
                    clipnorm=1.0,
                    clipvalue=0.5,
                ),
                metrics = metric_list,
            )
        else:
            metric_list = [
                tf.keras.metrics.BinaryAccuracy(),
                tf.keras.metrics.AUC(num_thresholds=10000)
            ]
            model.compile(
                loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3,
                    clipnorm=1.0,
                    clipvalue=0.5,
                ),
                metrics = metric_list,
            )
        return model

    APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
    if not os.path.exists(os.path.join(APP_DIR,"data")):
        os.makedirs(os.path.join(APP_DIR,"data"))

    def split_train_eval_test_dataset(self,dataset):
        """区分训练验证测试集
        """
        dataset_size = tf.data.experimental.cardinality(dataset).numpy()
        print("总共有数据{}条".format(dataset_size))
        dataset = dataset.shuffle(dataset_size,seed=1)
        train_size = int(0.6 * dataset_size)
        eval_size = int(0.2 * dataset_size)
        test_size = int(0.2 * dataset_size)

        train_dataset = dataset.take(train_size)
        test_dataset = dataset.skip(train_size)
        eval_dataset = test_dataset.skip(eval_size)
        test_dataset = test_dataset.take(test_size)
        return train_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
            eval_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
            test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    def split_train_eval_test_npa(self,x_npa,y_npa):
        x_train,x_test,y_train,y_test =  sklearn.model_selection.train_test_split(x_npa, y_npa, test_size=0.2, random_state=24)
        return x_train,y_train,x_test,y_test

    def build_model_callback(self):
        callback_path = os.path.join(APP_DIR,"model_callback","weights.{epoch:02d}-{val_loss:.2f}.hdf5")
        if not os.path.exists(os.path.dirname(callback_path)):
            os.makedirs(os.path.dirname(callback_path))

        model_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=callback_path,
            monitor='val_loss',
            verbose=1,
            save_best_only=False,
            save_weights_only=True,
            mode='auto',
            save_freq='epoch',
        )

        tf_board_dir = os.path.join(APP_DIR,"model_tensorboard")
        if not os.path.exists(os.path.dirname(tf_board_dir)):
             os.makedirs(os.path.dirname(tf_board_dir))

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tf_board_dir,histogram_freq=1,update_freq='batch')
        return model_callback,tensorboard_callback


    APP_DIR  = os.path.dirname(os.path.realpath('__file__'))

#     def trans_gensim_word2vec2tf_embedding(self,word2vector_file_path:str):
#         """把gensim的word2vec结果转化为tf.keras.layers.Embedding需要的结果
#         """

#         word2vec_model = gensim.models.Word2Vec.load(word2vector_file_path)

#         #所有的词
#         word_list = [word for word, word_info in word2vec_model.wv.vocab.items()]

#         #词到index的映射
#         word2index_dict = {"<PADDING>": 0, "<UNK>":1}

#         #保存特殊词的padding
#         specical_word_count = len(word2index_dict)

#         #词到词向量的映射
#         word2vector_dict = {}

#         #初始化embeddings_matrix

#         embeddings_matrix = np.zeros((len(word_list) + specical_word_count, word2vec_model.vector_size))
#         #初始化unk为-1,1分布
#         embeddings_matrix[word2index_dict["<UNK>"]] = (1 / np.sqrt(len(word_list) + specical_word_count) * (2 * np.random.rand(word2vec_model.vector_size) - 1))

#         for i,word in enumerate(word_list):
#             #从0开始
#             word_index = i + specical_word_count
#             word2index_dict[str(word)] = word_index
#             word2vector_dict[str(word)] = word2vec_model.wv[word] # 词语：词向量
#             embeddings_matrix[word_index] = word2vec_model.wv[word]  # 词向量矩阵

#         #写入文件
#         with open(os.path.join(APP_DIR,"data","word2index.json"),"w",encoding="utf8") as f:
#             json.dump(word2index_dict,f,ensure_ascii=False)

#         return embeddings_matrix,word2vector_dict,word2index_dict


#     def trans2index(self,word2index_dict,word):
#         """转换"""
#         if word in word2index_dict:
#             return word2index_dict[word]
#         else:
#             if "<UNK>" in word2index_dict:
#                 return word2index_dict["<UNK>"]
#             else:
#                 raise ValueError("没有这个值，请检查")


#     # def trans_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
#     #     """把data文件转化为tf.data
#     #     """

#     #     tag2index_dict = {}
#     #     tag_index_count = len(tag2index_dict)
#     #     lac = LAC.LAC(mode="seg")

#     #     df = pd.read_csv(data_file_path)

#     #     x_list = []
#     #     for doc in df["content"]:
#     #         word_list = lac.run(doc)
#     #         x_list.append([trans2index(word2index_dict,word) for word in word_list])
#     #     x_npa = np.array(x_list)

#     #     y_list = []
#     #     for tag in df["tag"]:
#     #         tag = tag.strip()
#     #         if not (tag in tag2index_dict):
#     #             tag2index_dict[tag] = tag_index_count
#     #             tag_index_count += 1
#     #         y_list.append(tag2index_dict[tag])
#     #     y_npa = np.array(y_list,dtype=np.uint8)

#     #     print("x_list[:1]:{}".format(x_list[:1]))
#     #     print("y_list[:1]:{}".format(y_list[:1]))

#     #     #写入文件
#     #     with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
#     #         json.dump(tag2index_dict,f,ensure_ascii=False)

#     #     if not x_max_length:
#     #         x_max_length0 = np.max(np.array([len(v) for v in x_list]))
#     #         x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
#     #         print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

#     #     x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

#     #     x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
#     #     print("x_npa[:1]:{}".format(x_npa[:1]))
#     #     print("y_npa[:1]:{}".format(y_npa[:1]))
#     #     print("x_npa.shape = {}".format(x_npa.shape))
#     #     print("y_npa.shape = {}".format(y_npa.shape))

#     #     return x_npa,y_npa,tag2index_dict



#     # def trans_tokenize_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
#     #     """把已经分好词的data文件转化为tf.data
#     #     """

#     #     tag2index_dict = {}
#     #     tag_index_count = len(tag2index_dict)
#     #     lac = LAC.LAC(mode="seg")

#     #     x_list = []
#     #     y_list = []
#     #     with open(data_file_path) as f:
#     #         for line in f:
#     #             temp_dict = json.loads(line.strip())
#     #             word_list = temp_dict["content_tokenize"]
#     #             tag = temp_dict["tag"].strip()
#     #             if not (tag in tag2index_dict):
#     #                 tag2index_dict[tag] = tag_index_count
#     #                 tag_index_count += 1
#     #             x_list.append([trans2index(word2index_dict,word) for word in word_list])
#     #             y_list.append(tag2index_dict[tag])
#     #     x_npa = np.array(x_list)
#     #     y_npa = np.array(y_list,dtype=np.uint8)

#     #     print("x_list[:1]:{}".format(x_list[:1]))
#     #     print("y_list[:1]:{}".format(y_list[:1]))

#     #     #写入文件
#     #     with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
#     #         json.dump(tag2index_dict,f,ensure_ascii=False)

#     #     if not x_max_length:
#     #         x_max_length0 = np.max(np.array([len(v) for v in x_list]))
#     #         x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
#     #         print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

#     #     x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

#     #     x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
#     #     print("x_npa[:1]:{}".format(x_npa[:1]))
#     #     print("y_npa[:1]:{}".format(y_npa[:1]))
#     #     print("x_npa.shape = {}".format(x_npa.shape))
#     #     print("y_npa.shape = {}".format(y_npa.shape))

#     #     return x_npa,y_npa,tag2index_dict


#     def trans_multi_input_tokenize_data2npa(self,data_file_path:str,x_max_length:int=None,word2index_dict=None):
#         """把已经分好词的data文件转化为tf.data , 多输入版本
#         """

#         tag2index_dict = {}
#         tag_index_count = len(tag2index_dict)

#         x_list = []
#         y_list = []
#         with open(data_file_path) as f:
#             for line in f:
#                 temp_dict = json.loads(line.strip())
#                 text_tokenize_list = temp_dict["all_content_tokenize"]
#                 tag = temp_dict["tag"].strip()
#                 if not (tag in tag2index_dict):
#                     tag2index_dict[tag] = tag_index_count
#                     tag_index_count += 1
#                 x_list.append([[a.trans2index(word2index_dict,word) for word in word_list] for word_list in text_tokenize_list])
#                 y_list.append(tag2index_dict[tag])
#         y_npa = np.array(y_list,dtype=np.uint8)

#         print("x_list[:1]:{}".format(x_list[:1]))
#         print("y_list[:1]:{}".format(y_list[:1]))

#         #写入文件
#         with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
#             json.dump(tag2index_dict,f,ensure_ascii=False)

#         if not x_max_length:
#             x_max_length0 = np.max(np.array([len(v) for v in x_list]))
#             x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
#             print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

#         for i in range(len(x_list)):
#             x_list[i] = tf.keras.preprocessing.sequence.pad_sequences(x_list[i],maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)
#         x_npa = np.array(x_list,dtype=np.int32)

#         x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
#         print("x_npa[:1]:{}".format(x_npa[:1]))
#         print("y_npa[:1]:{}".format(y_npa[:1]))
#         print("x_npa.shape = {}".format(x_npa.shape))
#         print("y_npa.shape = {}".format(y_npa.shape))

#         return x_npa,y_npa,tag2index_dict

    def train(self,input_number = 10,sentence_maxlen = 512,epochs = 7,batch_size = 64,learning_rate = 1e-3,
              is_embedding_training = False,embedding_dropout_rate = 0.2):
        #建模模型
        #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
        #with strategy.scope():
        model = a.build_model(
            input_number = input_number,
            sentence_maxlen = sentence_maxlen,
            vocab_size = vocab_size,
            tag_size = tag_size,
            embedding_dim = embedding_dim,
            embedding_matrix = embedding_matrix,
            is_embedding_training = is_embedding_training,
            embedding_dropout_rate = embedding_dropout_rate,
            learning_rate = learning_rate
        )

        #model_callback = build_model_callback()

        print(model.summary())

        history = model.fit(
            {"input{}".format(i):x_train_eval[:,i] for i in range(input_number)},
            y_train_eval,
            validation_split = 0.25,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            #callbacks=list(model_callback),
            #class_weight = class_weight_dict,
        )

        #测试集
        model.evaluate({"input{}".format(i):x_test[:,i] for i in range(input_number)},y_test)
        #保存模型
        print("即将保存模型：{}".format(model_path))
        model.save(model_path)


if __name__ == "__main__":

#     input_number = 10
#     #句子的最大长度
#     sentence_maxlen = 512
#     #训练次数
#     epochs = 7
#     #批大小
#     batch_size = 64
#     #学习率
#     learning_rate = 1e-3
#     # learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
#     #     1e-3, decay_steps=100, decay_rate=0.1, staircase=False
#     # )

#     #embedding layer是否参加训练
#     is_embedding_training = False
#     # is_embedding_training = True
#     #embedding的dropout比率
#     #embedding_dropout_rate = 0.35
#     embedding_dropout_rate = 0.2


    #数据文件
    #data_csv = os.path.join(APP_DIR,"data","最终数据集.csv")
    #data_jsonl = os.path.join(APP_DIR,"data","最终数据集tokenize.jsonl")
    #data_jsonl = "/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/age/最终数据集tokenize.jsonl"
    data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
    #word2vec路径
    #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
    word2vector_file_path = r"/home/zourui/data/dim256/word2vector.bin"
    #模型保存路径
    model_path = os.path.join(APP_DIR,"model_output1")
    ###以上是需要修改的部分
    a = Position_model()
    if not os.path.exists(os.path.dirname(model_path)):
        os.makedirs(os.path.dirname(model_path))

    #导入gensim的word2vector
    embedding_matrix,word2vector_dict,word2index_dict = a.trans_gensim_word2vec2tf_embedding(word2vector_file_path)
    vocab_size,embedding_dim = embedding_matrix.shape

    #划分训练集+验证集，测试集
    #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
    x_npa,y_npa,tag2index_dict = a.trans_multi_input_tokenize_data2npa(data_jsonl,sentence_maxlen,word2index_dict)
    class_weight_dict = {tag:np.sqrt(len(y_npa)/number) for tag,number in enumerate(np.bincount(y_npa))}
    x_train_eval,y_train_eval,x_test,y_test = a.split_train_eval_test_npa(x_npa,y_npa)
#     print(x_train_eval[:3])
#     print(y_train_eval[:3])
#     print(class_weight_dict)
#     print(collections.Counter(y_train_eval))
#     print(collections.Counter(y_test))
    tag_size = len(tag2index_dict)
    
#      #建模模型
#     #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
#     #with strategy.scope():
#     model = build_model(
#         input_number = input_number,
#         sentence_maxlen = sentence_maxlen,
#         vocab_size = vocab_size,
#         tag_size = tag_size,
#         embedding_dim = embedding_dim,
#         embedding_matrix = embedding_matrix,
#         is_embedding_training = is_embedding_training,
#         embedding_dropout_rate = embedding_dropout_rate,
#         learning_rate = learning_rate
#     )

#     #model_callback = build_model_callback()

#     print(model.summary())

#     history = model.fit(
#         {"input{}".format(i):x_train_eval[:,i] for i in range(input_number)},
#         y_train_eval,
#         validation_split = 0.25,
#         batch_size=batch_size,
#         epochs=epochs,
#         verbose=1,
#         #callbacks=list(model_callback),
#         #class_weight = class_weight_dict,
#     )

#     #测试集
#     model.evaluate({"input{}".format(i):x_test[:,i] for i in range(input_number)},y_test)
#     #保存模型
#     print("即将保存模型：{}".format(model_path))
#     model.save(model_path)

x_list[:1]:[[[64145, 60511, 11, 32049, 1674, 6, 129, 0, 31305, 9802, 2, 1, 11, 22136, 7621, 1518, 2, 28247, 341, 54, 12126, 11, 1006, 11786, 39276, 11, 1, 18, 1, 20828, 15290, 8973, 692, 11, 244, 341, 740, 8909, 112, 67716, 4256, 98, 11, 463, 2732, 129, 1583, 15550, 24570, 11, 21143, 19024, 389, 55, 14236, 57, 18, 11367, 54, 60511, 8, 14236, 6205, 11, 281, 1669, 19591, 245, 2897, 341, 740, 4256, 18, 40, 244, 20139, 11, 21560, 1036, 54, 5066, 274, 17043, 1380, 103, 8, 2361, 1, 18, 15357, 281, 128, 5649, 102, 20681, 8, 11, 54, 10361, 1333, 8479, 8, 33166, 18, 60511, 514, 71010, 76381, 8, 13568, 11, 32049, 4197, 6227, 12447, 3898, 6725, 103, 8, 291, 11, 991, 7631, 2585, 5203, 8, 2065, 18, 2789, 11367, 28422, 14236, 8, 15550, 24570, 2793, 11, 4155, 4450, 1507, 8, 6924, 1763, 11, 32049, 12552, 388, 3595, 1583, 2993, 2, 21421, 281, 883, 128, 6725, 103, 8, 1, 11, 991, 5203, 2502, 1, 60511, 11, 6, 281, 21560, 24034, 291, 1, 1, 18, 32049, 535, 12793, 3613, 50327, 11, 5649, 14521, 535, 5221, 151

In [42]:
a.train()

这是一个多分类模型
Model: "multi_textcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input1 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input3 (InputLayer)             [(None, 512)]        0                                            
____________________________________________________________________________

KeyboardInterrupt: 

In [2]:
import os
import time
import collections
import sklearn.model_selection
import gensim
import numpy as np
import tensorflow as tf
#from utils import trans_gensim_word2vec2tf_embedding,trans_data2tf_data,trans_tokenize_data2tf_data,trans_multi_input_tokenize_data2npa
#from model_multi_textcnn import build_model

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
if not os.path.exists(os.path.join(APP_DIR,"data")):
    os.makedirs(os.path.join(APP_DIR,"data"))

# def split_train_eval_test_dataset(dataset):
#     """区分训练验证测试集
#     """
#     dataset_size = tf.data.experimental.cardinality(dataset).numpy()
#     print("总共有数据{}条".format(dataset_size))
#     dataset = dataset.shuffle(dataset_size,seed=1)
#     train_size = int(0.6 * dataset_size)
#     eval_size = int(0.2 * dataset_size)
#     test_size = int(0.2 * dataset_size)

#     train_dataset = dataset.take(train_size)
#     test_dataset = dataset.skip(train_size)
#     eval_dataset = test_dataset.skip(eval_size)
#     test_dataset = test_dataset.take(test_size)
#     return train_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
#         eval_dataset.prefetch(tf.data.experimental.AUTOTUNE), \
#         test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# def split_train_eval_test_npa(x_npa,y_npa):
#     x_train,x_test,y_train,y_test =  sklearn.model_selection.train_test_split(x_npa, y_npa, test_size=0.2, random_state=24)
#     return x_train,y_train,x_test,y_test

# def build_model_callback():
#     callback_path = os.path.join(APP_DIR,"model_callback","weights.{epoch:02d}-{val_loss:.2f}.hdf5")
#     if not os.path.exists(os.path.dirname(callback_path)):
#         os.makedirs(os.path.dirname(callback_path))

#     model_callback = tf.keras.callbacks.ModelCheckpoint(
#         filepath=callback_path,
#         monitor='val_loss',
#         verbose=1,
#         save_best_only=False,
#         save_weights_only=True,
#         mode='auto',
#         save_freq='epoch',
#     )

#     tf_board_dir = os.path.join(APP_DIR,"model_tensorboard")
#     if not os.path.exists(os.path.dirname(tf_board_dir)):
#          os.makedirs(os.path.dirname(tf_board_dir))

#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tf_board_dir,histogram_freq=1,update_freq='batch')
#     return model_callback,tensorboard_callback


if __name__ == "__main__":

    input_number = 10
    #句子的最大长度
    sentence_maxlen = 512
    #训练次数
#     epochs = 7
#     #批大小
#     batch_size = 64
#     #学习率
#     learning_rate = 1e-3
#     # learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
#     #     1e-3, decay_steps=100, decay_rate=0.1, staircase=False
#     # )

#     #embedding layer是否参加训练
#     is_embedding_training = False
#     # is_embedding_training = True
#     #embedding的dropout比率
#     #embedding_dropout_rate = 0.35
#     embedding_dropout_rate = 0.2


    #数据文件
    #data_csv = os.path.join(APP_DIR,"data","最终数据集.csv")
    #data_jsonl = os.path.join(APP_DIR,"data","最终数据集tokenize.jsonl")
    #data_jsonl = "/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/age/最终数据集tokenize.jsonl"
    data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
    #word2vec路径
    #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
    word2vector_file_path = r"/home/zourui/data/dim256/word2vector.bin"
    #模型保存路径
    #model_path = os.path.join(APP_DIR,"model_output2")
    ###以上是需要修改的部分

#     if not os.path.exists(os.path.dirname(model_path)):
#         os.makedirs(os.path.dirname(model_path))

    #导入gensim的word2vector
    embedding_matrix,word2vector_dict,word2index_dict = trans_gensim_word2vec2tf_embedding(word2vector_file_path)
    vocab_size,embedding_dim = embedding_matrix.shape

    #划分训练集+验证集，测试集
    #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
    x_npa,y_npa,tag2index_dict = trans_multi_input_tokenize_data2npa(data_jsonl,sentence_maxlen,word2index_dict)
    class_weight_dict = {tag:np.sqrt(len(y_npa)/number) for tag,number in enumerate(np.bincount(y_npa))}
#    x_train_eval,y_train_eval,x_test,y_test = split_train_eval_test_npa(x_npa,y_npa)
  #  print(x_train_eval[:3])
   # print(y_train_eval[:3])
  #  print(class_weight_dict)
  #  print(collections.Counter(y_train_eval))
   # print(collections.Counter(y_test))
    tag_size = len(tag2index_dict)   

x_list[:1]:[[[64145, 60511, 11, 32049, 1674, 6, 129, 0, 31305, 9802, 2, 1, 11, 22136, 7621, 1518, 2, 28247, 341, 54, 12126, 11, 1006, 11786, 39276, 11, 1, 18, 1, 20828, 15290, 8973, 692, 11, 244, 341, 740, 8909, 112, 67716, 4256, 98, 11, 463, 2732, 129, 1583, 15550, 24570, 11, 21143, 19024, 389, 55, 14236, 57, 18, 11367, 54, 60511, 8, 14236, 6205, 11, 281, 1669, 19591, 245, 2897, 341, 740, 4256, 18, 40, 244, 20139, 11, 21560, 1036, 54, 5066, 274, 17043, 1380, 103, 8, 2361, 1, 18, 15357, 281, 128, 5649, 102, 20681, 8, 11, 54, 10361, 1333, 8479, 8, 33166, 18, 60511, 514, 71010, 76381, 8, 13568, 11, 32049, 4197, 6227, 12447, 3898, 6725, 103, 8, 291, 11, 991, 7631, 2585, 5203, 8, 2065, 18, 2789, 11367, 28422, 14236, 8, 15550, 24570, 2793, 11, 4155, 4450, 1507, 8, 6924, 1763, 11, 32049, 12552, 388, 3595, 1583, 2993, 2, 21421, 281, 883, 128, 6725, 103, 8, 1, 11, 991, 5203, 2502, 1, 60511, 11, 6, 281, 21560, 24034, 291, 1, 1, 18, 32049, 535, 12793, 3613, 50327, 11, 5649, 14521, 535, 5221, 151

In [4]:
import json
import gensim
import os
import LAC
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn.utils

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))

def trans_gensim_word2vec2tf_embedding(word2vector_file_path:str):
    """把gensim的word2vec结果转化为tf.keras.layers.Embedding需要的结果
    """

    word2vec_model = gensim.models.Word2Vec.load(word2vector_file_path)

    #所有的词
    word_list = [word for word, word_info in word2vec_model.wv.vocab.items()]

    #词到index的映射
    word2index_dict = {"<PADDING>": 0, "<UNK>":1}

    #保存特殊词的padding
    specical_word_count = len(word2index_dict)

    #词到词向量的映射
    word2vector_dict = {}

    #初始化embeddings_matrix

    embeddings_matrix = np.zeros((len(word_list) + specical_word_count, word2vec_model.vector_size))
    #初始化unk为-1,1分布
    embeddings_matrix[word2index_dict["<UNK>"]] = (1 / np.sqrt(len(word_list) + specical_word_count) * (2 * np.random.rand(word2vec_model.vector_size) - 1))

    for i,word in enumerate(word_list):
        #从0开始
        word_index = i + specical_word_count
        word2index_dict[str(word)] = word_index
        word2vector_dict[str(word)] = word2vec_model.wv[word] # 词语：词向量
        embeddings_matrix[word_index] = word2vec_model.wv[word]  # 词向量矩阵

    #写入文件
    with open(os.path.join(APP_DIR,"data","word2index.json"),"w",encoding="utf8") as f:
        json.dump(word2index_dict,f,ensure_ascii=False)

    return embeddings_matrix,word2vector_dict,word2index_dict


def trans2index(word2index_dict,word):
    """转换"""
    if word in word2index_dict:
        return word2index_dict[word]
    else:
        if "<UNK>" in word2index_dict:
            return word2index_dict["<UNK>"]
        else:
            raise ValueError("没有这个值，请检查")


# def trans_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
#     """把data文件转化为tf.data
#     """

#     tag2index_dict = {}
#     tag_index_count = len(tag2index_dict)
#     lac = LAC.LAC(mode="seg")

#     df = pd.read_csv(data_file_path)

#     x_list = []
#     for doc in df["content"]:
#         word_list = lac.run(doc)
#         x_list.append([trans2index(word2index_dict,word) for word in word_list])
#     x_npa = np.array(x_list)

#     y_list = []
#     for tag in df["tag"]:
#         tag = tag.strip()
#         if not (tag in tag2index_dict):
#             tag2index_dict[tag] = tag_index_count
#             tag_index_count += 1
#         y_list.append(tag2index_dict[tag])
#     y_npa = np.array(y_list,dtype=np.uint8)

#     print("x_list[:1]:{}".format(x_list[:1]))
#     print("y_list[:1]:{}".format(y_list[:1]))

#     #写入文件
#     with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
#         json.dump(tag2index_dict,f,ensure_ascii=False)

#     if not x_max_length:
#         x_max_length0 = np.max(np.array([len(v) for v in x_list]))
#         x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
#         print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

#     x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

#     x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
#     print("x_npa[:1]:{}".format(x_npa[:1]))
#     print("y_npa[:1]:{}".format(y_npa[:1]))
#     print("x_npa.shape = {}".format(x_npa.shape))
#     print("y_npa.shape = {}".format(y_npa.shape))

#     return x_npa,y_npa,tag2index_dict



# def trans_tokenize_data2tf_data(data_file_path:str,x_max_length:int=None,word2index_dict=None):
#     """把已经分好词的data文件转化为tf.data
#     """

#     tag2index_dict = {}
#     tag_index_count = len(tag2index_dict)
#     lac = LAC.LAC(mode="seg")

#     x_list = []
#     y_list = []
#     with open(data_file_path) as f:
#         for line in f:
#             temp_dict = json.loads(line.strip())
#             word_list = temp_dict["content_tokenize"]
#             tag = temp_dict["tag"].strip()
#             if not (tag in tag2index_dict):
#                 tag2index_dict[tag] = tag_index_count
#                 tag_index_count += 1
#             x_list.append([trans2index(word2index_dict,word) for word in word_list])
#             y_list.append(tag2index_dict[tag])
#     x_npa = np.array(x_list)
#     y_npa = np.array(y_list,dtype=np.uint8)

#     print("x_list[:1]:{}".format(x_list[:1]))
#     print("y_list[:1]:{}".format(y_list[:1]))

#     #写入文件
#     with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
#         json.dump(tag2index_dict,f,ensure_ascii=False)

#     if not x_max_length:
#         x_max_length0 = np.max(np.array([len(v) for v in x_list]))
#         x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
#         print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

#     x_npa = tf.keras.preprocessing.sequence.pad_sequences(x_npa,maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

#     x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
#     print("x_npa[:1]:{}".format(x_npa[:1]))
#     print("y_npa[:1]:{}".format(y_npa[:1]))
#     print("x_npa.shape = {}".format(x_npa.shape))
#     print("y_npa.shape = {}".format(y_npa.shape))

#     return x_npa,y_npa,tag2index_dict


def trans_multi_input_tokenize_data2npa(data_file_path:str,x_max_length:int=None,word2index_dict=None):
    """把已经分好词的data文件转化为tf.data , 多输入版本
    """

    tag2index_dict = {}
    tag_index_count = len(tag2index_dict)

    x_list = []
    y_list = []
    with open(data_file_path) as f:
        for line in f:
            temp_dict = json.loads(line.strip())
            text_tokenize_list = temp_dict["all_content_tokenize"]
            tag = temp_dict["tag"].strip()
            if not (tag in tag2index_dict):
                tag2index_dict[tag] = tag_index_count
                tag_index_count += 1
            x_list.append([[trans2index(word2index_dict,word) for word in word_list] for word_list in text_tokenize_list])
            y_list.append(tag2index_dict[tag])
    y_npa = np.array(y_list,dtype=np.uint8)

    print("x_list[:1]:{}".format(x_list[:1]))
    print("y_list[:1]:{}".format(y_list[:1]))

    #写入文件
    with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
        json.dump(tag2index_dict,f,ensure_ascii=False)

    if not x_max_length:
        x_max_length0 = np.max(np.array([len(v) for v in x_list]))
        x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
        print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))
    
    for i in range(len(x_list)):
        x_list[i] = tf.keras.preprocessing.sequence.pad_sequences(x_list[i],maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)
    x_npa = np.array(x_list,dtype=np.int32)

    x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
    print("x_npa[:1]:{}".format(x_npa[:1]))
    print("y_npa[:1]:{}".format(y_npa[:1]))
    print("x_npa.shape = {}".format(x_npa.shape))
    print("y_npa.shape = {}".format(y_npa.shape))

    return x_npa,y_npa,tag2index_dict


In [2]:
import json
import gensim
import os
import LAC
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn.utils
import time
import collections
import sklearn.model_selection

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
class Number_Transform(object):
    
    def trans_gensim_word2vec2tf_embedding(self,word2vector_file_path):
        """把gensim的word2vec结果转化为tf.keras.layers.Embedding需要的结果
        """
        word2vec_model = gensim.models.Word2Vec.load(word2vector_file_path)
        #所有的词
        word_list = [word for word, word_info in word2vec_model.wv.vocab.items()]
        #词到index的映射
        word2index_dict = {"<PADDING>": 0, "<UNK>":1}
        #保存特殊词的padding
        specical_word_count = len(word2index_dict)
        #词到词向量的映射
        word2vector_dict = {}
        #初始化embeddings_matrix
        embeddings_matrix = np.zeros((len(word_list) + specical_word_count, word2vec_model.vector_size))
        #初始化unk为-1,1分布
        embeddings_matrix[word2index_dict["<UNK>"]] = (1 / np.sqrt(len(word_list) + specical_word_count) * (2 * np.random.rand(word2vec_model.vector_size) - 1))
        for i,word in enumerate(word_list):
            #从0开始
            word_index = i + specical_word_count
            word2index_dict[str(word)] = word_index
            word2vector_dict[str(word)] = word2vec_model.wv[word] # 词语：词向量
            embeddings_matrix[word_index] = word2vec_model.wv[word]  # 词向量矩阵
        #写入文件
        with open(os.path.join(APP_DIR,"data","word2index.json"),"w",encoding="utf8") as f:
            json.dump(word2index_dict,f,ensure_ascii=False)
        return embeddings_matrix,word2vector_dict,word2index_dict


    def trans2index(self,word2index_dict,word):
        """转换"""
        if word in word2index_dict:
            return word2index_dict[word]
        else:
            if "<UNK>" in word2index_dict:
                return word2index_dict["<UNK>"]
            else:
                raise ValueError("没有这个值，请检查")

    def trans_multi_input_tokenize_data2npa(self,data_file_path,x_max_length,word2index_dict):
        """把已经分好词的data文件转化为tf.data , 多输入版本
        """
        tag2index_dict = {}
        tag_index_count = len(tag2index_dict)
        x_list = []
        y_list = []
        with open(data_file_path) as f:
            for line in f:
                temp_dict = json.loads(line.strip())
                text_tokenize_list = temp_dict["all_content_tokenize"]
                tag = temp_dict["tag"].strip()
                if not (tag in tag2index_dict):
                    tag2index_dict[tag] = tag_index_count
                    tag_index_count += 1
                x_list.append([[Number_Transform.trans2index(self,word2index_dict,word) for word in word_list] for word_list in text_tokenize_list])
                y_list.append(tag2index_dict[tag])
        y_npa = np.array(y_list,dtype=np.uint8)

      #  print("x_list[:1]:{}".format(x_list[:1]))
      #  print("y_list[:1]:{}".format(y_list[:1]))

        #写入文件
        with open(os.path.join(APP_DIR,"data/tag2index.json"),"w",encoding="utf8") as f:
            json.dump(tag2index_dict,f,ensure_ascii=False)

        if not x_max_length:
            x_max_length0 = np.max(np.array([len(v) for v in x_list]))
            x_max_length = int(np.max(np.percentile(np.array([len(v) for v in x_list]),99.7)))
            print("数据集中最长的句子长度为:{},设定的最长的句子长度为:{}".format(x_max_length0,x_max_length))

        for i in range(len(x_list)):
            x_list[i] = tf.keras.preprocessing.sequence.pad_sequences(x_list[i],maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)
        x_npa = np.array(x_list,dtype=np.int32)

        x_npa,y_npa = sklearn.utils.shuffle(x_npa,y_npa,random_state=0)
    #     print("x_npa[:1]:{}".format(x_npa[:1]))
    #     print("y_npa[:1]:{}".format(y_npa[:1]))
    #     print("x_npa.shape = {}".format(x_npa.shape))
    #     print("y_npa.shape = {}".format(y_npa.shape))

        return x_npa,y_npa,tag2index_dict

    def out_x_y(self,input_number,sentence_maxlen,data_jsonl,word2vector_file_path):

        input_number = input_number
        #句子的最大长度
        sentence_maxlen = sentence_maxlen

       # data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
        #word2vec路径
        #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
        word2vector_file_path = word2vector_file_path

        #导入gensim的word2vector
        embedding_matrix,word2vector_dict,word2index_dict = Number_Transform.trans_gensim_word2vec2tf_embedding(self,word2vector_file_path)
        vocab_size,embedding_dim = embedding_matrix.shape

        #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
        x_npa,y_npa,tag2index_dict = Number_Transform.trans_multi_input_tokenize_data2npa(self,data_jsonl,sentence_maxlen,word2index_dict)
        class_weight_dict = {tag:np.sqrt(len(y_npa)/number) for tag,number in enumerate(np.bincount(y_npa))}
        tag_size = len(tag2index_dict)
        print('转换完毕！')
        return x_npa,y_npa

In [3]:
data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
word2vector_file_path = r"/home/zourui/data/dim256/word2vector.bin"
input_number = 10
sentence_maxlen = 512
nt = Number_Transform()
x_npa,y_npa = nt.out_x_y(input_number,sentence_maxlen,data_jsonl,word2vector_file_path)

转换完毕！


In [2]:
import os
import time
import collections
import sklearn.model_selection
import gensim
import numpy as np
import tensorflow as tf
#from utils import trans_gensim_word2vec2tf_embedding,trans_data2tf_data,trans_tokenize_data2tf_data,trans_multi_input_tokenize_data2npa
#from model_multi_textcnn import build_model

APP_DIR  = os.path.dirname(os.path.realpath('__file__'))
if not os.path.exists(os.path.join(APP_DIR,"data")):
    os.makedirs(os.path.join(APP_DIR,"data"))

if __name__ == "__main__":

    input_number = 10
    #句子的最大长度
    sentence_maxlen = 512

    data_jsonl = r"/mnt/d/zourui/predict_user_attribute20201214/age/raw_data/user_profile/position/最终数据集tokenize8.jsonl"
    #word2vec路径
    #word2vector_file_path = os.path.join(APP_DIR,"data","word2vector.bin")
    word2vector_file_path = r"/home/zourui/data/dim256/word2vector.bin"

    #导入gensim的word2vector
    embedding_matrix,word2vector_dict,word2index_dict = trans_gensim_word2vec2tf_embedding(word2vector_file_path)
    vocab_size,embedding_dim = embedding_matrix.shape

    #x_npa,y_npa,tag2index_dict = trans_data2tf_data(data_csv,sentence_maxlen,word2index_dict)
    x_npa,y_npa,tag2index_dict = trans_multi_input_tokenize_data2npa(data_jsonl,sentence_maxlen,word2index_dict)
    class_weight_dict = {tag:np.sqrt(len(y_npa)/number) for tag,number in enumerate(np.bincount(y_npa))}
    tag_size = len(tag2index_dict)

x_list[:1]:[[[64145, 60511, 11, 32049, 1674, 6, 129, 0, 31305, 9802, 2, 1, 11, 22136, 7621, 1518, 2, 28247, 341, 54, 12126, 11, 1006, 11786, 39276, 11, 1, 18, 1, 20828, 15290, 8973, 692, 11, 244, 341, 740, 8909, 112, 67716, 4256, 98, 11, 463, 2732, 129, 1583, 15550, 24570, 11, 21143, 19024, 389, 55, 14236, 57, 18, 11367, 54, 60511, 8, 14236, 6205, 11, 281, 1669, 19591, 245, 2897, 341, 740, 4256, 18, 40, 244, 20139, 11, 21560, 1036, 54, 5066, 274, 17043, 1380, 103, 8, 2361, 1, 18, 15357, 281, 128, 5649, 102, 20681, 8, 11, 54, 10361, 1333, 8479, 8, 33166, 18, 60511, 514, 71010, 76381, 8, 13568, 11, 32049, 4197, 6227, 12447, 3898, 6725, 103, 8, 291, 11, 991, 7631, 2585, 5203, 8, 2065, 18, 2789, 11367, 28422, 14236, 8, 15550, 24570, 2793, 11, 4155, 4450, 1507, 8, 6924, 1763, 11, 32049, 12552, 388, 3595, 1583, 2993, 2, 21421, 281, 883, 128, 6725, 103, 8, 1, 11, 991, 5203, 2502, 1, 60511, 11, 6, 281, 21560, 24034, 291, 1, 1, 18, 32049, 535, 12793, 3613, 50327, 11, 5649, 14521, 535, 5221, 151

In [4]:
sklearn.__version__

'0.23.2'

In [5]:
y_npa = pd.DataFrame(y_npa)
y_npa.columns = ['position']
x_npa = x_npa.reshape(13134,5120)
x_npa = pd.DataFrame(x_npa)
x_npa

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,73413,64116,21779,0,13097,4563,27162,1380,5936,9802,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,475,6,80083,485,64138,8,5850,2,272,125,...,0,0,0,0,0,0,0,0,0,0
2,346,50,11,272,4761,55,62032,57,11,991,...,1294,128,1583,1,8,21732,1303,4837,11,1651
3,1363,102,55,1,57,8,15596,0,1,5936,...,112,38075,112,15846,11,54,1006,467,26613,2258
4,14791,1,55,15538,57,272,325,0,31305,1518,...,11,155,40016,8,46715,4,135,2468,5442,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,44825,17315,55,13888,13888,57,11,378,2348,1406,...,1081,111,12,112,887,112,1685,237,8,483
13130,5394,1010,535,1006,55,1850,14271,57,11,341,...,14796,109,3664,44425,1181,111,313,1,109,1
13131,2361,15057,5886,55,1,57,325,1,818,66220,...,0,0,0,0,0,0,0,0,0,0
13132,55,1,44274,13056,57,2432,11,2440,60109,7000,...,244,22001,11,10996,80,81,1,82,22443,24917


In [6]:
position_data = pd.merge(y_npa,x_npa,on = y_npa.index)
position_data = position_data.drop('key_0',axis=1)

In [7]:
position_data

,position,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,73413,64116,21779,0,13097,4563,27162,1380,5936,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,1,475,6,80083,485,64138,8,5850,2,272,...,0,0,0,0,0,0,0,0,0,0
2,1,346,50,11,272,4761,55,62032,57,11,...,1294,128,1583,1,8,21732,1303,4837,11,1651
3,2,1363,102,55,1,57,8,15596,0,1,...,112,38075,112,15846,11,54,1006,467,26613,2258
4,1,14791,1,55,15538,57,272,325,0,31305,...,11,155,40016,8,46715,4,135,2468,5442,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,1,44825,17315,55,13888,13888,57,11,378,2348,...,1081,111,12,112,887,112,1685,237,8,483
13130,3,5394,1010,535,1006,55,1850,14271,57,11,...,14796,109,3664,44425,1181,111,313,1,109,1
13131,4,2361,15057,5886,55,1,57,325,1,818,...,0,0,0,0,0,0,0,0,0,0
13132,1,55,1,44274,13056,57,2432,11,2440,60109,...,244,22001,11,10996,80,81,1,82,22443,24917


In [8]:
temp_0 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==0)
temp_1 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==1)
temp_2 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==2)
temp_3 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==3)
temp_4 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==4)
temp_5 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==5)
temp_6 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==6)

In [9]:
temp_0 = temp_0.drop('position',axis = 1)
temp_1 = temp_1.drop('position',axis = 1)
temp_2 = temp_2.drop('position',axis = 1)
temp_3 = temp_3.drop('position',axis = 1)
temp_4 = temp_4.drop('position',axis = 1)
temp_5 = temp_5.drop('position',axis = 1)
temp_6 = temp_6.drop('position',axis = 1)

In [10]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_0)
labels = model.predict(temp_0)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_00 = pd.merge(labels,temp_0,on = labels.index)
temp_00 = temp_00.drop('key_0',axis=1)
temp_000 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==0)
temp_000 = temp_000['position']
temp_end_0 = pd.merge(temp_000,temp_00,on = temp_00.index)
temp_end_0 = temp_end_0.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_0['b']))
b = max(a,key = a.get)
temp_end_0 = temp_end_0.groupby(temp_end_0.index).filter(lambda x:float(x['b'])==b)
temp_end_0

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
4,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
5,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,0,1,91353,43210,1325,2224,8,8315,0,1,...,39481,48399,15370,11,535,154,9114,166,11,103
563,0,1,3534,55,836,57,535,942,8,5886,...,9034,17476,495,610,11,38938,1674,129,1622,8
565,0,1,15218,2255,60199,1711,1,11,64483,3369,...,11,260,196,845,3347,18,538,1,3645,893
566,0,1,57058,6496,1323,1771,55,51025,57,6496,...,67,11,3132,3383,3146,8,5061,16721,2679,1675


In [11]:
model = KMeans(n_clusters=2)
model.fit(temp_1)
labels = model.predict(temp_1)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_11 = pd.merge(labels,temp_1,on = labels.index)
temp_11 = temp_11.drop('key_0',axis=1)
temp_111 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==1)
temp_111 = temp_111['position']
temp_end_1 = pd.merge(temp_111,temp_11,on = temp_11.index)
temp_end_1 = temp_end_1.drop('key_0',axis = 1)
temp_end_1.groupby('b').count()
a = dict(collections.Counter(temp_end_1['b']))
b = max(a,key = a.get)
temp_end_1 = temp_end_1.groupby(temp_end_1.index).filter(lambda x:float(x['b'])==b)
temp_end_1

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,1,1,475,6,80083,485,64138,8,5850,2,...,0,0,0,0,0,0,0,0,0,0
1,1,1,346,50,11,272,4761,55,62032,57,...,1294,128,1583,1,8,21732,1303,4837,11,1651
2,1,1,14791,1,55,15538,57,272,325,0,...,11,155,40016,8,46715,4,135,2468,5442,18
3,1,1,1,0,13097,4563,27162,1,9802,4563,...,8115,483,1,495,67,11,1,10132,8,1
4,1,1,4738,523,245,111031,11,5558,12684,245,...,11,1,54,515,5558,54190,83,44824,11,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6376,1,1,55,1,57,1,409,876,0,55,...,347,8,54,1675,325,4548,4628,11,2679,14258
6377,1,1,1016,8523,1,2,5389,10,1467,1675,...,17137,15218,266,983,1,899,1326,1094,18,85
6379,1,1,321,244,35404,322,179,274,396,8,...,389,1,2,1,11,1006,483,1,112,88616
6380,1,1,44825,17315,55,13888,13888,57,11,378,...,1081,111,12,112,887,112,1685,237,8,483


In [12]:
model = KMeans(n_clusters=2)
model.fit(temp_2)
labels = model.predict(temp_2)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_22 = pd.merge(labels,temp_2,on = labels.index)
temp_22 = temp_22.drop('key_0',axis=1)
temp_222 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==2)
temp_222 = temp_222['position']
temp_end_2 = pd.merge(temp_222,temp_22,on = temp_22.index)
temp_end_2 = temp_end_2.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_2['b']))
b = max(a,key = a.get)
temp_end_2 = temp_end_2.groupby(temp_end_2.index).filter(lambda x:float(x['b'])==b)
temp_end_2

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,2,1,1363,102,55,1,57,8,15596,0,...,112,38075,112,15846,11,54,1006,467,26613,2258
1,2,1,15220,11005,129,464,40910,11,4554,30268,...,11,991,3053,47752,934,1,40,1669,5374,244
2,2,1,1,112,3533,112,352,2951,2,2345,...,1,35451,18,3198,106330,883,3286,2206,8,1
3,2,1,272,5904,2134,2762,53851,325,0,1562,...,67,11,1484,774,23601,6954,3291,4255,6954,4750
4,2,1,1015,272,535,1845,55,1176,57,129,...,54,1,1,8,483,1,495,11,185,3192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802,2,1,2581,11,54,13916,535,61316,47852,19181,...,741,2570,12470,463,55303,50263,1,18,244,1
2803,2,1,57058,6496,1323,1771,55,51025,57,6496,...,8,5500,1964,110511,2177,14499,111,18,6235,7852
2804,2,1,55,1,57,8,86903,1181,11,1481,...,0,0,0,0,0,0,0,0,0,0
2805,2,1,362,12120,8,102,55,24148,57,2,...,28457,8,810,11,9352,126,3559,10103,11,1553


In [13]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_3)
labels = model.predict(temp_3)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_33 = pd.merge(labels,temp_3,on = labels.index)
temp_33 = temp_33.drop('key_0',axis=1)
temp_333 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==3)
temp_333 = temp_333['position']
temp_end_3 = pd.merge(temp_333,temp_33,on = temp_33.index)
temp_end_3 = temp_end_3.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_3['b']))
b = max(a,key = a.get)
temp_end_3 = temp_end_3.groupby(temp_end_3.index).filter(lambda x:float(x['b'])==b)
temp_end_3

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,3,1,112143,8547,1,11,19596,1,1002,362,...,2460,2348,16988,129,483,1,109,65082,112,38642
1,3,1,15220,11005,129,464,40910,11,4554,30268,...,3793,7589,38968,41699,62527,37957,1,18,57,11176
3,3,1,26018,2298,1577,8,5282,6190,126,11,...,472,185,40,1,18,244,1,8,9329,1
4,3,1,57058,6496,1323,1771,55,51025,57,6496,...,1460,119821,11,2255,1076,153,229,81,1563,18
6,3,1,55,1,57,8,15449,11,8626,892,...,4835,244,34,50,54,3719,1688,8,93460,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,3,1,2263,6493,1674,55,1,57,325,0,...,14814,20637,1562,110107,11,1069,3591,1,899,1
1619,3,1,1765,39375,423,32399,2,1952,2267,11,...,11,1,3955,1,40,271,2,55,5299,565
1621,3,1,1,2406,892,948,32918,11,234,3428,...,3793,7589,38968,41699,62527,37957,1,18,57,11176
1623,3,1,1015,11,378,265,129,378,8,1,...,16899,2377,11,4144,58067,1232,15416,104868,2068,2143


In [14]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_4)
labels = model.predict(temp_4)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_44 = pd.merge(labels,temp_4,on = labels.index)
temp_44 = temp_44.drop('key_0',axis=1)
temp_444 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==4)
temp_444 = temp_444['position']
temp_end_4 = pd.merge(temp_444,temp_44,on = temp_44.index)
temp_end_4 = temp_end_4.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_4['b']))
b = max(a,key = a.get)
temp_end_4 = temp_end_4.groupby(temp_end_4.index).filter(lambda x:float(x['b'])==b)
temp_end_4

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,4,0,5306,38649,1,8,1,11,12166,1518,...,311,1,11,1597,716,12,1283,8,1,11
1,4,0,53496,8,2196,12865,2,272,415,1,...,1,18,599,483,7673,495,244,1211,67,8605
3,4,0,2263,3914,1231,185,751,3891,26775,26776,...,82,2616,11,441,38935,244,657,9512,237,8
4,4,0,279,1711,2494,11,3698,1711,6207,2,...,0,0,0,0,0,0,0,0,0,0
5,4,0,42356,8,55,16077,15426,57,11,37,...,126,2364,526,244,6738,10862,674,850,38,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,4,0,57058,6496,1323,1771,55,51025,57,6496,...,0,0,0,0,0,0,0,0,0,0
339,4,0,2986,54,296,297,8,1376,72219,0,...,311,1,11,1597,716,12,1283,8,1,11
340,4,0,71511,20300,4885,34383,6340,2,2120,1808,...,23189,129,81466,14447,3314,18,55,1047,22524,1582
341,4,0,4760,535,43890,2229,2,2263,717,11883,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_5)
labels = model.predict(temp_5)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_55 = pd.merge(labels,temp_5,on = labels.index)
temp_55 = temp_55.drop('key_0',axis=1)
temp_555 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==5)
temp_555 = temp_555['position']
temp_end_5 = pd.merge(temp_555,temp_55,on = temp_55.index)
temp_end_5 = temp_end_5.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_5['b']))
b = max(a,key = a.get)
temp_end_5 = temp_end_5.groupby(temp_end_5.index).filter(lambda x:float(x['b'])==b)
temp_end_5

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,5,0,57058,6496,1323,1771,55,51025,57,6496,...,5203,1327,55,1808,1,57,540,45,107,9847
1,5,0,2288,4846,11,15871,54775,5335,15712,8,...,244,3232,3233,8,165,248,11,1630,1340,51281
2,5,0,378,244,2613,50,1674,9017,2,55,...,3849,25845,55,1,57,11,1,2278,1,11009
3,5,0,3014,752,237,23810,112,1050,11,2827,...,11,5306,8,10744,2679,1,18,4155,11,1
4,5,0,4837,63929,975,12968,3701,11,260,1865,...,5203,1327,55,1808,1,57,540,45,107,9847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,5,0,281,36710,9912,1669,4490,1,25807,8,...,543,37,1,8,65026,11,192,7433,8,2576
1130,5,0,6866,21960,55,1,57,12863,0,31305,...,11,1324,154,10304,1336,21794,1010,22,8419,72765
1131,5,0,6969,11,1840,1006,2726,17043,325,0,...,3545,8,1,3920,82,5053,7731,11,2853,4467
1132,5,0,1,2,1583,6519,11,13631,13102,0,...,1291,109,77,111,11,8284,689,5585,8,30032


In [16]:
#聚类找异常值
model = KMeans(n_clusters=2)
model.fit(temp_6)
labels = model.predict(temp_6)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_66 = pd.merge(labels,temp_6,on = labels.index)
temp_66 = temp_66.drop('key_0',axis=1)
temp_666 = position_data.groupby(position_data.index).filter(lambda x : float(x['position'])==6)
temp_666 = temp_666['position']
temp_end_6 = pd.merge(temp_666,temp_66,on = temp_66.index)
temp_end_6 = temp_end_6.drop('key_0',axis = 1)
a = dict(collections.Counter(temp_end_5['b']))
b = max(a,key = a.get)
temp_end_6 = temp_end_6.groupby(temp_end_6.index).filter(lambda x:float(x['b'])==b)
temp_end_6

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,6,0,46328,92794,514,73463,325,346,1762,1588,...,34,12653,50,9485,81,6760,1375,11,336,7123
1,6,0,45169,38809,1763,0,55,4713,11602,11,...,1,18,599,483,7673,495,244,1211,67,8605
2,6,0,81793,2134,29781,31252,2134,11,6663,272,...,0,0,0,0,0,0,0,0,0,0
3,6,0,13788,102,30207,30233,13246,948,39393,650,...,24623,21081,11,991,1,47943,11,40,5944,244
4,6,0,5374,1774,7206,1,34,732,11,65661,...,65729,28581,6023,11,20826,1974,18,429,414,1702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,6,0,1326,54,321,1964,55,4590,57,2703,...,79541,2732,18,378,20931,4450,7653,18,378,11367
271,6,0,21794,68756,55,25679,57,244,3920,1763,...,249,15449,7822,4416,24104,244,4142,2248,1,3420
272,6,0,66654,786,325,29629,13861,8866,2598,6412,...,4523,11,1630,5990,2720,1583,5751,1481,56159,18
273,6,0,750,3712,8,12383,3006,54,1675,325,...,8,527,57,18,1537,11,7490,1481,15748,11


In [17]:
x_npa1 = pd.concat([temp_end_0, temp_end_1], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_2], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_3], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_4], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_5], axis=0, ignore_index=True)
x_npa1 = pd.concat([x_npa1, temp_end_6], axis=0, ignore_index=True)
x_npa1

,position,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,6,0,1326,54,321,1964,55,4590,57,2703,...,79541,2732,18,378,20931,4450,7653,18,378,11367
11878,6,0,21794,68756,55,25679,57,244,3920,1763,...,249,15449,7822,4416,24104,244,4142,2248,1,3420
11879,6,0,66654,786,325,29629,13861,8866,2598,6412,...,4523,11,1630,5990,2720,1583,5751,1481,56159,18
11880,6,0,750,3712,8,12383,3006,54,1675,325,...,8,527,57,18,1537,11,7490,1481,15748,11


In [18]:
x_npa1 = x_npa1.drop('b',axis = 1)
x_npa1

,position,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,73413,64116,21779,0,13097,4563,27162,1380,5936,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,5304,68859,8,55,48915,57,2,1,822,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,47256,11551,2470,2,7827,15813,11,9310,311,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,3712,1,1,2,2986,1588,54,248,2853,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,238,44,12779,1774,586,6929,325,3533,112,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,6,1326,54,321,1964,55,4590,57,2703,5306,...,79541,2732,18,378,20931,4450,7653,18,378,11367
11878,6,21794,68756,55,25679,57,244,3920,1763,11,...,249,15449,7822,4416,24104,244,4142,2248,1,3420
11879,6,66654,786,325,29629,13861,8866,2598,6412,70144,...,4523,11,1630,5990,2720,1583,5751,1481,56159,18
11880,6,750,3712,8,12383,3006,54,1675,325,0,...,8,527,57,18,1537,11,7490,1481,15748,11


In [19]:
x_data = x_npa1.drop('position',axis = 1)
y_data = x_npa1['position']

In [20]:
ss = StandardScaler()
x_data = ss.fit_transform(x_data)
x_data = pd.DataFrame(x_data)
x_data

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,2.924969,3.073260,0.752454,-0.480162,0.249183,-0.201926,1.095525,-0.407545,-0.144874,0.037442,...,-0.234935,-0.070254,-0.263520,-0.416174,-0.433443,-0.174471,-0.400720,-0.177328,-0.284635,-0.157485
1,-0.364179,3.338564,-0.481780,-0.476866,2.380966,-0.495061,-0.489934,-0.498806,-0.465233,0.011059,...,-0.084389,-0.315562,-0.427641,-0.309810,-0.433904,-0.404960,-0.361640,-0.382623,-0.374844,0.291175
2,1.661784,0.132978,-0.342205,-0.480042,-0.064472,0.529935,-0.489408,0.117254,-0.497244,0.734856,...,-0.411338,-0.012827,-0.425765,2.182458,-0.423166,-0.332604,-0.246841,-0.241896,-0.340901,-0.349327
3,-0.441060,-0.513084,-0.482176,-0.480042,-0.352595,-0.395463,-0.486898,-0.482459,-0.338004,-0.498723,...,-0.415424,-0.097565,-0.430697,-0.416174,3.976402,-0.408634,-0.401458,-0.279699,6.045400,-0.413729
4,-0.608828,-0.510679,0.242229,-0.373856,-0.495436,-0.048008,-0.471078,-0.265062,-0.509710,-0.529619,...,-0.226571,-0.386590,0.754539,-0.338417,-0.323689,-0.408634,-0.040143,-0.293772,-0.351471,-0.414231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,-0.556286,-0.510119,-0.464035,-0.362470,-0.527039,-0.200170,-0.486723,-0.319990,-0.184339,-0.529619,...,4.661700,-0.239730,-0.430211,-0.393983,0.944934,-0.075572,0.163821,-0.384486,-0.357052,0.400185
11878,0.432163,3.332803,-0.479115,1.058638,-0.526920,-0.482896,-0.261221,-0.382198,-0.516037,-0.128378,...,-0.400676,0.446646,0.111810,-0.156299,1.153967,-0.390939,-0.095548,-0.230652,-0.381803,-0.169748
11879,2.598560,-0.469174,-0.463808,1.295339,0.294654,0.078003,-0.338393,-0.074532,3.877347,2.214965,...,-0.127804,-0.386590,-0.318250,-0.063650,-0.254780,-0.290540,0.023314,-0.283562,3.305197,-0.413729
11880,-0.584102,-0.305505,-0.481780,0.261883,-0.351404,-0.495256,-0.392273,-0.477364,-0.516726,1.281551,...,-0.416063,-0.358740,-0.427502,-0.415173,-0.332715,-0.408409,0.151780,-0.283562,0.652051,-0.414231


In [20]:
estimator = SVC(kernel = "linear")
selector = RFE(estimator,n_features_to_select = 200,step = 128)
selector = selector.fit(x_data,y_data)
rank = selector.ranking_
rank = pd.DataFrame(rank)

In [21]:
rank.columns = ['rank']
rank.to_csv('/home/zourui/data/predict_user_attribute20200911/raw_data/age/rank/position_rank.csv',index = False)

In [21]:
rank = pd.read_csv('/home/zourui/data/predict_user_attribute20200911/raw_data/age/rank/position_rank.csv')

In [22]:
x_T = x_data.T

In [23]:
rank_data = pd.merge(rank,x_T,on = rank.index)
rank_data = rank_data.drop('key_0',axis = 1)
rank_data

,rank,0,1,2,3,4,5,6,7,8,...,11872,11873,11874,11875,11876,11877,11878,11879,11880,11881
0,21,2.924969,-0.364179,1.661784,-0.441060,-0.608828,-0.400688,-0.386731,0.485816,-0.080944,...,-0.620273,-0.571305,-0.620273,0.204319,-0.620273,-0.556286,0.432163,2.598560,-0.584102,-0.549477
1,3,3.073260,3.338564,0.132978,-0.513084,-0.510679,-0.510119,3.062799,-0.499436,0.889290,...,0.308338,-0.497925,0.256821,-0.075328,-0.351261,-0.510119,3.332803,-0.469174,-0.305505,-0.505924
2,34,0.752454,-0.481780,-0.342205,-0.482176,0.242229,-0.434952,-0.426959,2.589887,-0.168161,...,-0.482233,-0.451903,-0.481780,-0.428489,-0.479172,-0.464035,-0.479115,-0.463808,-0.481780,0.325906
3,25,-0.480162,-0.476866,-0.480042,-0.480042,-0.373856,0.007803,0.296938,-0.480162,0.566598,...,1.395772,-0.369601,-0.168015,4.415064,-0.300089,-0.362470,1.058638,1.295339,0.261883,1.139955
4,25,0.249183,2.380966,-0.064472,-0.352595,-0.495436,0.385774,-0.310040,1.332870,-0.510970,...,-0.439966,-0.527039,-0.530253,-0.529658,1.442262,-0.527039,-0.526920,0.294654,-0.351404,-0.530253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,5,-0.174471,-0.404960,-0.332604,-0.408634,-0.408634,-0.154151,1.296788,-0.408334,-0.409234,...,-0.405110,-0.407884,-0.404960,-0.400836,-0.298113,-0.075572,-0.390939,-0.290540,-0.408409,1.764889
5116,7,-0.400720,-0.361640,-0.246841,-0.401458,-0.040143,-0.398060,-0.278016,0.280024,-0.401532,...,0.794996,-0.361788,-0.361640,-0.401458,-0.392002,0.163821,-0.095548,0.023314,0.151780,-0.401458
5117,9,-0.177328,-0.382623,-0.241896,-0.279699,-0.293772,0.686763,-0.384486,-0.384486,-0.385727,...,-0.366412,-0.385658,-0.382623,-0.385658,-0.326263,-0.384486,-0.230652,-0.283562,-0.283562,-0.385175
5118,29,-0.284635,-0.374844,-0.340901,6.045400,-0.351471,-0.283388,-0.236839,0.102330,-0.381869,...,-0.378126,-0.142560,-0.374844,-0.346416,-0.381147,-0.357052,-0.381803,3.305197,0.652051,-0.188124


In [24]:
rank_data_200 = rank_data.groupby(rank_data.index).filter(lambda x : float(x['rank'])==1)
rank_data_200 = rank_data_200.drop('rank',axis = 1)
rank_data_200 = rank_data_200.T

In [25]:
all_data = pd.merge(y_data,rank_data_200,on = y_data.index)
all_data = all_data.drop('key_0',axis = 1)
all_data

,position,11,34,69,74,90,140,155,182,201,...,4879,4899,4929,4957,5025,5027,5028,5034,5048,5104
0,0,-0.532670,-0.461405,-0.391201,-0.294919,-0.122935,-0.442809,-0.151833,-0.388940,-0.427510,...,-0.228909,0.096380,-0.004375,3.718615,-0.400705,-0.415991,-0.385584,3.590275,-0.385347,-0.388032
1,0,-0.527347,-0.455876,-0.409182,-0.426788,-0.421671,1.707684,-0.429419,-0.301274,-0.406118,...,-0.419811,0.084052,-0.362562,-0.438753,-0.397531,-0.051812,-0.322192,-0.443158,-0.382892,-0.326540
2,0,-0.212996,-0.364480,-0.398166,-0.383660,-0.426004,-0.442809,-0.292524,0.391351,-0.262642,...,1.044505,-0.386528,-0.072413,-0.429868,-0.400705,-0.284066,-0.382397,3.356945,-0.362566,4.649172
3,0,-0.532210,-0.468026,-0.264133,2.158851,0.387490,2.098589,-0.408270,-0.403983,-0.271573,...,-0.411443,-0.387232,0.024686,0.226291,-0.378411,0.218731,0.469343,-0.387856,-0.318307,-0.387612
4,0,-0.532736,-0.381339,-0.412094,-0.365507,-0.406569,-0.428681,-0.068744,0.289487,-0.253782,...,0.135521,-0.386528,-0.371905,-0.447280,-0.352869,6.352194,0.127926,-0.446238,0.923074,-0.321980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,6,-0.511376,0.257958,-0.288319,-0.436673,-0.361835,-0.442809,2.314569,-0.285988,0.398847,...,-0.420520,-0.386528,-0.228378,-0.447280,-0.400188,-0.219003,1.410594,-0.446696,-0.293378,-0.389552
11878,6,-0.532210,4.841427,0.967041,-0.415803,-0.419887,-0.314932,3.712683,0.044615,-0.432695,...,-0.111049,-0.248242,-0.354824,-0.447280,-0.374794,-0.414768,-0.399717,-0.446041,-0.386651,-0.276058
11879,6,-0.351407,-0.459289,-0.412094,-0.429909,0.138458,-0.436685,2.582274,-0.403983,-0.372842,...,-0.416620,-0.269305,0.165806,3.063101,-0.301636,-0.318541,-0.333069,-0.416424,-0.367705,-0.075962
11880,6,0.860647,0.052094,-0.318772,-0.436500,-0.281033,-0.216585,-0.145747,-0.387248,-0.351522,...,-0.415840,-0.114465,-0.290168,0.254237,-0.271370,1.258224,0.250415,0.137774,-0.344540,-0.389395


In [26]:
data_x = all_data.drop('position',axis = 1)
data_y = all_data['position']

In [27]:
x_train_svm, x_test_svm, y_train_svm, y_test_svm = train_test_split(data_x, data_y, random_state=155)

In [32]:
svm = SVC(C = 10,kernel = 'linear')
svm.fit(x_train_svm, y_train_svm) 
#lr.fit(x_train,y_train)
preds = svm.predict(x_test_svm)
print('准确率为%f' %((preds==y_test_svm).sum()/float(y_test_svm.shape[0])))
#joblib.dump(svm, '../code/model_svm')

准确率为0.489398


In [32]:
preds.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [33]:
a = pd.DataFrame(preds)

In [34]:
a.columns = ['position']

In [35]:
a['position'].value_counts()

1    2993
Name: position, dtype: int64

In [ ]:
def svm(C):
    val = cross_val_score(
        SVC(C=C
           ),
        x_train_svm, y_train_svm, scoring='accuracy',cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (100,500)
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 5,n_iter =5)
params_1 = svm_bo
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
print(t_end - t_start)
svm_classifier=SVC(C=L[0]['C'],kernel='linear')

# 拟合模型
svm_classifier.fit(x_train_svm, y_train_svm)

# 使用模型预测
preds = svm_classifier.predict(x_test_svm)

print('准确率为%f' %((preds==y_test_svm).sum()/float(y_test_svm.shape[0])))
#joblib.dump(svm_classifier, '../code/model_svm')

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.4515   |  471.9    |
|  2        |  0.4515   |  311.4    |
|  3        |  0.4515   |  301.9    |
|  4        |  0.4515   |  416.4    |
|  5        |  0.4515   |  450.1    |
|  6        |  0.4515   |  100.0    |
|  7        |  0.4515   |  100.1    |
|  8        |  0.4515   |  100.1    |
|  9        |  0.4515   |  500.0    |
|  10       |  0.4515   |  100.1    |
{'C': 471.9232854934403}
0.4515175730669626
2800.8100554943085


In [27]:
from sklearn.svm import OneClassSVM
classer=OneClassSVM()
classer.fit(temp_0)
yHat=classer.predict(temp_0)
yHat = pd.DataFrame(yHat)
print(yHat)

     0
0    1
1   -1
2   -1
3    1
4   -1
..  ..
563 -1
564  1
565 -1
566  1
567  1

[568 rows x 1 columns]


In [38]:
yHat.columns = ['b']
temp_00 = pd.merge(yHat,temp_0,on = yHat.index)

In [42]:
#temp_00 = temp_00.drop('key_0',axis=1)
temp_00.groupby('b').count()

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
-1,287,287,287,287,287,287,287,287,287,287,...,287,287,287,287,287,287,287,287,287,287
1,281,281,281,281,281,281,281,281,281,281,...,281,281,281,281,281,281,281,281,281,281


In [43]:
classer=OneClassSVM()
classer.fit(temp_1)
yHat_1=classer.predict(temp_1)
yHat_1 = pd.DataFrame(yHat_1)
print(yHat_1)

      0
0     1
1    -1
2    -1
3    -1
4     1
...  ..
6377 -1
6378 -1
6379  1
6380  1
6381 -1

[6382 rows x 1 columns]


In [44]:
yHat_1.columns = ['b']
temp_11 = pd.merge(yHat_1,temp_1,on = yHat_1.index)
temp_11 = temp_11.drop('key_0',axis=1)
temp_11.groupby('b').count()

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
-1,3189,3189,3189,3189,3189,3189,3189,3189,3189,3189,...,3189,3189,3189,3189,3189,3189,3189,3189,3189,3189
1,3193,3193,3193,3193,3193,3193,3193,3193,3193,3193,...,3193,3193,3193,3193,3193,3193,3193,3193,3193,3193


In [14]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_0)
labels = model.predict(temp_0)
labels = pd.DataFrame(labels)

In [15]:
labels.columns = ['b']
temp_00 = pd.merge(labels,temp_0,on = labels.index)
temp_00 = temp_00.drop('key_0',axis=1)

In [16]:
temp_00.groupby('b').count()

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,40,40,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40
1,528,528,528,528,528,528,528,528,528,528,...,528,528,528,528,528,528,528,528,528,528


In [17]:
temp_000 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==0)
temp_000 = temp_000['a']

In [18]:
temp_end_0 = pd.merge(temp_000,temp_00,on = temp_00.index)
temp_end_0 = temp_end_0.drop('key_0',axis = 1)

In [19]:
temp_end_0 = temp_end_0.groupby(temp_end_0.index).filter(lambda x:float(x['b'])==1)
temp_end_0

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
4,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
5,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,0,1,3534,55,836,57,535,942,8,5886,...,9034,17476,495,610,11,38938,1674,129,1622,8
564,0,1,13770,389,20441,1589,4843,1582,6769,1,...,1,244,1,8547,88181,819,11,35982,8,47217
565,0,1,15218,2255,60199,1711,1,11,64483,3369,...,11,260,196,845,3347,18,538,1,3645,893
566,0,1,57058,6496,1323,1771,55,51025,57,6496,...,67,11,3132,3383,3146,8,5061,16721,2679,1675


In [21]:
#聚类普通企业员工
model = KMeans(n_clusters=2)
model.fit(temp_1)
labels = model.predict(temp_1)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_11 = pd.merge(labels,temp_1,on = labels.index)
temp_11 = temp_11.drop('key_0',axis=1)
temp_111 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==1)
temp_111 = temp_111['a']
temp_end_1 = pd.merge(temp_111,temp_11,on = temp_11.index)
temp_end_1 = temp_end_1.drop('key_0',axis = 1)
temp_end_1 = temp_end_1.groupby(temp_end_1.index).filter(lambda x:float(x['b'])==1)
temp_end_1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,1,1,475,6,80083,485,64138,8,5850,2,...,0,0,0,0,0,0,0,0,0,0
1,1,1,346,50,11,272,4761,55,62032,57,...,1294,128,1583,1,8,21732,1303,4837,11,1651
2,1,1,14791,1,55,15538,57,272,325,0,...,11,155,40016,8,46715,4,135,2468,5442,18
3,1,1,1,0,13097,4563,27162,1,9802,4563,...,8115,483,1,495,67,11,1,10132,8,1
4,1,1,4738,523,245,111031,11,5558,12684,245,...,11,1,54,515,5558,54190,83,44824,11,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6376,1,1,55,1,57,1,409,876,0,55,...,347,8,54,1675,325,4548,4628,11,2679,14258
6377,1,1,1016,8523,1,2,5389,10,1467,1675,...,17137,15218,266,983,1,899,1326,1094,18,85
6379,1,1,321,244,35404,322,179,274,396,8,...,389,1,2,1,11,1006,483,1,112,88616
6380,1,1,44825,17315,55,13888,13888,57,11,378,...,1081,111,12,112,887,112,1685,237,8,483


In [24]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_2)
labels = model.predict(temp_2)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_22 = pd.merge(labels,temp_2,on = labels.index)
temp_22 = temp_22.drop('key_0',axis=1)
temp_222 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==2)
temp_222 = temp_222['a']
temp_end_2 = pd.merge(temp_222,temp_22,on = temp_22.index)
temp_end_2 = temp_end_2.drop('key_0',axis = 1)


In [25]:
temp_end_2.groupby('b').count()

,a,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,97,97,97,97,97,97,97,97,97,97,...,97,97,97,97,97,97,97,97,97,97
1,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710,...,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710


In [26]:
temp_end_2 = temp_end_2.groupby(temp_end_2.index).filter(lambda x:float(x['b'])==1)
temp_end_2

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,2,1,1363,102,55,1,57,8,15596,0,...,112,38075,112,15846,11,54,1006,467,26613,2258
1,2,1,15220,11005,129,464,40910,11,4554,30268,...,11,991,3053,47752,934,1,40,1669,5374,244
2,2,1,1,112,3533,112,352,2951,2,2345,...,1,35451,18,3198,106330,883,3286,2206,8,1
3,2,1,272,5904,2134,2762,53851,325,0,1562,...,67,11,1484,774,23601,6954,3291,4255,6954,4750
4,2,1,1015,272,535,1845,55,1176,57,129,...,54,1,1,8,483,1,495,11,185,3192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802,2,1,2581,11,54,13916,535,61316,47852,19181,...,741,2570,12470,463,55303,50263,1,18,244,1
2803,2,1,57058,6496,1323,1771,55,51025,57,6496,...,8,5500,1964,110511,2177,14499,111,18,6235,7852
2804,2,1,55,1,57,8,86903,1181,11,1481,...,0,0,0,0,0,0,0,0,0,0
2805,2,1,362,12120,8,102,55,24148,57,2,...,28457,8,810,11,9352,126,3559,10103,11,1553


In [29]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_3)
labels = model.predict(temp_3)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_33 = pd.merge(labels,temp_3,on = labels.index)
temp_33 = temp_33.drop('key_0',axis=1)
temp_333 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==3)
temp_333 = temp_333['a']
temp_end_3 = pd.merge(temp_333,temp_33,on = temp_33.index)
temp_end_3 = temp_end_3.drop('key_0',axis = 1)

In [30]:
temp_end_3.groupby('b').count()

,a,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,157,157,157,157,157,157,157,157,157,157,...,157,157,157,157,157,157,157,157,157,157
1,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,...,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468


In [31]:
temp_end_3 = temp_end_3.groupby(temp_end_3.index).filter(lambda x:float(x['b'])==1)
temp_end_3

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,3,1,112143,8547,1,11,19596,1,1002,362,...,2460,2348,16988,129,483,1,109,65082,112,38642
1,3,1,15220,11005,129,464,40910,11,4554,30268,...,3793,7589,38968,41699,62527,37957,1,18,57,11176
3,3,1,26018,2298,1577,8,5282,6190,126,11,...,472,185,40,1,18,244,1,8,9329,1
4,3,1,57058,6496,1323,1771,55,51025,57,6496,...,1460,119821,11,2255,1076,153,229,81,1563,18
6,3,1,55,1,57,8,15449,11,8626,892,...,4835,244,34,50,54,3719,1688,8,93460,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,3,1,2263,6493,1674,55,1,57,325,0,...,14814,20637,1562,110107,11,1069,3591,1,899,1
1619,3,1,1765,39375,423,32399,2,1952,2267,11,...,11,1,3955,1,40,271,2,55,5299,565
1621,3,1,1,2406,892,948,32918,11,234,3428,...,3793,7589,38968,41699,62527,37957,1,18,57,11176
1623,3,1,1015,11,378,265,129,378,8,1,...,16899,2377,11,4144,58067,1232,15416,104868,2068,2143


In [34]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_4)
labels = model.predict(temp_4)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_44 = pd.merge(labels,temp_4,on = labels.index)
temp_44 = temp_44.drop('key_0',axis=1)
temp_444 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==4)
temp_444 = temp_444['a']
temp_end_4 = pd.merge(temp_444,temp_44,on = temp_44.index)
temp_end_4 = temp_end_4.drop('key_0',axis = 1)

In [35]:
temp_end_4.groupby('b').count()

,a,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,325,325,325,325,325,325,325,325,325,325,...,325,325,325,325,325,325,325,325,325,325
1,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18


In [36]:
temp_end_4 = temp_end_4.groupby(temp_end_4.index).filter(lambda x:float(x['b'])==0)
temp_end_4

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
1,4,0,53496,8,2196,12865,2,272,415,1,...,1,18,599,483,7673,495,244,1211,67,8605
2,4,0,1,0,13097,4563,27162,1,9802,4563,...,0,0,0,0,0,0,0,0,0,0
3,4,0,2263,3914,1231,185,751,3891,26775,26776,...,82,2616,11,441,38935,244,657,9512,237,8
4,4,0,279,1711,2494,11,3698,1711,6207,2,...,0,0,0,0,0,0,0,0,0,0
5,4,0,42356,8,55,16077,15426,57,11,37,...,126,2364,526,244,6738,10862,674,850,38,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,4,0,8909,732,13041,11,1,5602,8,54,...,4973,390,82,34189,188,82,5859,362,6920,5374
338,4,0,57058,6496,1323,1771,55,51025,57,6496,...,0,0,0,0,0,0,0,0,0,0
340,4,0,71511,20300,4885,34383,6340,2,2120,1808,...,23189,129,81466,14447,3314,18,55,1047,22524,1582
341,4,0,4760,535,43890,2229,2,2263,717,11883,...,0,0,0,0,0,0,0,0,0,0


In [39]:
#聚类
model = KMeans(n_clusters=2)
model.fit(temp_5)
labels = model.predict(temp_5)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_55 = pd.merge(labels,temp_5,on = labels.index)
temp_55 = temp_55.drop('key_0',axis=1)
temp_555 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==5)
temp_555 = temp_555['a']
temp_end_5 = pd.merge(temp_555,temp_55,on = temp_55.index)
temp_end_5 = temp_end_5.drop('key_0',axis = 1)

In [40]:
temp_end_5.groupby('b').count()

,a,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,1118,1118,1118,1118,1118,1118,1118,1118,1118,1118,...,1118,1118,1118,1118,1118,1118,1118,1118,1118,1118
1,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16


In [41]:
temp_end_5 = temp_end_5.groupby(temp_end_5.index).filter(lambda x:float(x['b'])==0)
temp_end_5

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,5,0,57058,6496,1323,1771,55,51025,57,6496,...,5203,1327,55,1808,1,57,540,45,107,9847
1,5,0,2288,4846,11,15871,54775,5335,15712,8,...,244,3232,3233,8,165,248,11,1630,1340,51281
2,5,0,378,244,2613,50,1674,9017,2,55,...,3849,25845,55,1,57,11,1,2278,1,11009
4,5,0,4837,63929,975,12968,3701,11,260,1865,...,5203,1327,55,1808,1,57,540,45,107,9847
5,5,0,63408,322,34342,325,1,66936,2502,23980,...,17137,15218,266,983,1,899,1326,1094,18,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,5,0,281,36710,9912,1669,4490,1,25807,8,...,543,37,1,8,65026,11,192,7433,8,2576
1130,5,0,6866,21960,55,1,57,12863,0,31305,...,11,1324,154,10304,1336,21794,1010,22,8419,72765
1131,5,0,6969,11,1840,1006,2726,17043,325,0,...,3545,8,1,3920,82,5053,7731,11,2853,4467
1132,5,0,1,2,1583,6519,11,13631,13102,0,...,1291,109,77,111,11,8284,689,5585,8,30032


In [42]:
#聚类找异常值
model = KMeans(n_clusters=2)
model.fit(temp_6)
labels = model.predict(temp_6)
labels = pd.DataFrame(labels)
labels.columns = ['b']
temp_66 = pd.merge(labels,temp_6,on = labels.index)
temp_66 = temp_66.drop('key_0',axis=1)
temp_666 = position_total.groupby(position_total.index).filter(lambda x : float(x['a'])==6)
temp_666 = temp_666['a']
temp_end_6 = pd.merge(temp_666,temp_66,on = temp_66.index)
temp_end_6 = temp_end_6.drop('key_0',axis = 1)

In [43]:
temp_end_6.groupby('b').count()

,a,0,1,2,3,4,5,6,7,8,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
b,,,,,,,,,,,,,,,,,,,,,
0,267,267,267,267,267,267,267,267,267,267,...,267,267,267,267,267,267,267,267,267,267
1,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [44]:
temp_end_6 = temp_end_6.groupby(temp_end_6.index).filter(lambda x:float(x['b'])==0)
temp_end_6

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,6,0,46328,92794,514,73463,325,346,1762,1588,...,34,12653,50,9485,81,6760,1375,11,336,7123
1,6,0,45169,38809,1763,0,55,4713,11602,11,...,1,18,599,483,7673,495,244,1211,67,8605
2,6,0,81793,2134,29781,31252,2134,11,6663,272,...,0,0,0,0,0,0,0,0,0,0
3,6,0,13788,102,30207,30233,13246,948,39393,650,...,24623,21081,11,991,1,47943,11,40,5944,244
4,6,0,5374,1774,7206,1,34,732,11,65661,...,65729,28581,6023,11,20826,1974,18,429,414,1702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,6,0,1326,54,321,1964,55,4590,57,2703,...,79541,2732,18,378,20931,4450,7653,18,378,11367
271,6,0,21794,68756,55,25679,57,244,3920,1763,...,249,15449,7822,4416,24104,244,4142,2248,1,3420
272,6,0,66654,786,325,29629,13861,8866,2598,6412,...,4523,11,1630,5990,2720,1583,5751,1481,56159,18
273,6,0,750,3712,8,12383,3006,54,1675,325,...,8,527,57,18,1537,11,7490,1481,15748,11


In [45]:
x_npa1 = pd.concat([temp_end_0, temp_end_1], axis=0, ignore_index=True)

In [46]:
x_npa1 = pd.concat([x_npa1, temp_end_2], axis=0, ignore_index=True)
x_npa1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,2,1,2581,11,54,13916,535,61316,47852,19181,...,741,2570,12470,463,55303,50263,1,18,244,1
9004,2,1,57058,6496,1323,1771,55,51025,57,6496,...,8,5500,1964,110511,2177,14499,111,18,6235,7852
9005,2,1,55,1,57,8,86903,1181,11,1481,...,0,0,0,0,0,0,0,0,0,0
9006,2,1,362,12120,8,102,55,24148,57,2,...,28457,8,810,11,9352,126,3559,10103,11,1553


In [47]:
x_npa1 = pd.concat([x_npa1, temp_end_3], axis=0, ignore_index=True)
x_npa1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,3,1,2263,6493,1674,55,1,57,325,0,...,14814,20637,1562,110107,11,1069,3591,1,899,1
10472,3,1,1765,39375,423,32399,2,1952,2267,11,...,11,1,3955,1,40,271,2,55,5299,565
10473,3,1,1,2406,892,948,32918,11,234,3428,...,3793,7589,38968,41699,62527,37957,1,18,57,11176
10474,3,1,1015,11,378,265,129,378,8,1,...,16899,2377,11,4144,58067,1232,15416,104868,2068,2143


In [48]:
x_npa1 = pd.concat([x_npa1, temp_end_4], axis=0, ignore_index=True)
x_npa1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10796,4,0,8909,732,13041,11,1,5602,8,54,...,4973,390,82,34189,188,82,5859,362,6920,5374
10797,4,0,57058,6496,1323,1771,55,51025,57,6496,...,0,0,0,0,0,0,0,0,0,0
10798,4,0,71511,20300,4885,34383,6340,2,2120,1808,...,23189,129,81466,14447,3314,18,55,1047,22524,1582
10799,4,0,4760,535,43890,2229,2,2263,717,11883,...,0,0,0,0,0,0,0,0,0,0


In [49]:
x_npa1 = pd.concat([x_npa1, temp_end_5], axis=0, ignore_index=True)
x_npa1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11914,5,0,281,36710,9912,1669,4490,1,25807,8,...,543,37,1,8,65026,11,192,7433,8,2576
11915,5,0,6866,21960,55,1,57,12863,0,31305,...,11,1324,154,10304,1336,21794,1010,22,8419,72765
11916,5,0,6969,11,1840,1006,2726,17043,325,0,...,3545,8,1,3920,82,5053,7731,11,2853,4467
11917,5,0,1,2,1583,6519,11,13631,13102,0,...,1291,109,77,111,11,8284,689,5585,8,30032


In [50]:
x_npa1 = pd.concat([x_npa1, temp_end_6], axis=0, ignore_index=True)
x_npa1

,a,b,0,1,2,3,4,5,6,7,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
0,0,1,73413,64116,21779,0,13097,4563,27162,1380,...,2845,5872,2418,1,8,3131,11,3021,1481,3591
1,0,1,5304,68859,8,55,48915,57,2,1,...,5203,1327,55,1808,1,57,540,45,107,9847
2,0,1,47256,11551,2470,2,7827,15813,11,9310,...,82,6936,82,44149,164,1022,2094,2085,624,916
3,0,1,3712,1,1,2,2986,1588,54,248,...,18,5366,11,1,66947,8,1,1537,97896,18
4,0,1,238,44,12779,1774,586,6929,325,3533,...,2976,11,17076,1322,1674,8,4892,1333,463,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12181,6,0,1326,54,321,1964,55,4590,57,2703,...,79541,2732,18,378,20931,4450,7653,18,378,11367
12182,6,0,21794,68756,55,25679,57,244,3920,1763,...,249,15449,7822,4416,24104,244,4142,2248,1,3420
12183,6,0,66654,786,325,29629,13861,8866,2598,6412,...,4523,11,1630,5990,2720,1583,5751,1481,56159,18
12184,6,0,750,3712,8,12383,3006,54,1675,325,...,8,527,57,18,1537,11,7490,1481,15748,11


In [51]:
x_npa_end = x_npa1.drop(['b','a'],axis=1)
y_npa_end = x_npa1['a']

In [54]:
x_npa_end = np.array(x_npa_end)
np.shape(x_npa_end)

x_npa_end = x_npa_end.reshape(12186,10,512)

In [55]:
y_npa_end = np.array(y_npa_end)
np.shape(y_npa_end)

(12186,)

In [58]:
x_npa_svm = x_npa1.drop(['b','a'],axis = 1)
y_npa_svm = x_npa1['a']
ss = StandardScaler()
x_npa_svm = ss.fit_transform(x_npa_svm)
x_train_svm,y_train_svm,x_test_svm,y_test_svm = split_train_eval_test_npa(x_npa_svm,y_npa_svm)


def svm(C,gamma):
    val = cross_val_score(
        SVC(C=C,
            gamma = gamma),
        x_train_svm, y_train_svm, scoring='r2', cv=10).mean()
    return val
t_start = time.time()
params = {
    'C': (0.001,50),
    'gamma':(1,20),
    }
svm_bo = BayesianOptimization(svm,params)
svm_bo.maximize(init_points = 1,n_iter =1)
params = svm_bo.max
params = params["params"]
L = list()
L.append(params)
val = svm(**params)
print(params)
print(val)
t_end = time.time()
svm_classifier=SVC(C=L[0]['C'],gamma=L[0]['gamma'],kernel = 'rbf')

## 初始化模型
#xgb_classifier = xgb.XGBClassifier(n_estimators=20,\
     #                              max_depth=4, \
       #                            learning_rate=0.1, \
         #                          subsample=0.7, \
           #                        colsample_bytree=0.7)

# 拟合模型
svm_classifier.fit(x_train_svm, y_train_svm)

# 使用模型预测
preds = svm_classifier.predict(x_test_svm)

print('准确率为%f' %((preds==test_y).sum()/float(test_y.shape[0])))

# 模型存储
joblib.dump(svm_classifier, 'D:\huxiu\position\position_svm_model_data8.model')

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------


KeyboardInterrupt: 

In [ ]:
x_npa_svm = x_npa1.drop(['b','a'],axis = 1)
y_npa_svm = x_npa1['a']
ss = StandardScaler()
x_npa_svm = ss.fit_transform(x_npa_svm)
x_train_svm,y_train_svm,x_test_svm,y_test_svm = split_train_eval_test_npa(x_npa_svm,y_npa_svm)
svm = SVC(C = 15,
         gamma = 0.15,
         kernel = 'linear')
  #              x_train = np.array(x_train)
 #               x_test = np.array(x_test)
svm.fit(x_train_svm, y_train_svm)
               # validation_error.append(np.mean((xgboost.predict(x_test) - y_test) ** 2) ** 0.5)
    #            y_test = np.array(y_test)
#validation_error.append(np.mean((RF.predict(x_test)!= y_test).sum()/float(y_test.shape[0]))) 
pre_x_test = svm.predict(x_test_svm)
print('准确率为%f' %((pre_x_test==y_test_svm).sum()/float(y_test_svm.shape[0])))

In [179]:
x_train_end,y_train_end,x_test_end,y_test_end = split_train_eval_test_npa(x_npa_end,y_npa_end)

In [196]:
model = build_model(
        input_number = input_number,
        sentence_maxlen = sentence_maxlen,
        vocab_size = vocab_size,
        tag_size = tag_size,
        embedding_dim = embedding_dim,
        embedding_matrix = embedding_matrix,
        is_embedding_training = is_embedding_training,
        embedding_dropout_rate = embedding_dropout_rate,
        learning_rate = learning_rate
    )

    #model_callback = build_model_callback()

print(model.summary())

history = model.fit(
    {"input{}".format(i):x_train_end[:,i] for i in range(input_number)},
    y_train_end,
    validation_split = 0.25,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
        #callbacks=list(model_callback),
        #class_weight = class_weight_dict,
    )

#测试集
model.evaluate({"input{}".format(i):x_test_end[:,i] for i in range(input_number)},y_test_end)
#保存模型
print("即将保存模型：{}".format(model_path))
model.save(model_path)

这是一个多分类模型
Model: "multi_textcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input1 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 512)]        0                                            
__________________________________________________________________________________________________
input3 (InputLayer)             [(None, 512)]        0                                            
____________________________________________________________________________